In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 55em; }</style>"))
%matplotlib notebook
import numpy as np
import sympy as sp
import cxroots as cx
import scipy.special as spsp
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import matplotlib.cm as cm
from ipywidgets import AppLayout
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML, Javascript, FileLink
import traceback
from fpdf import FPDF
from urllib.request import urlopen
import os
from os import path
from PIL import Image
from matplotlib.offsetbox import (OffsetImage,AnchoredOffsetbox)
from module import *
from datetime import datetime

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code of P3delta Online is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
# -----------------------------------------------------------------------------
# GLOBAL VARIABLE
# -----------------------------------------------------------------------------

a_coeff_mid_generic = []
b_coeff_mid_generic = []
a_coeff_mid_co = []
b_coeff_mid_co = []
a_coeff_crrid_generic = []
b_coeff_crrid_generic = []

computedtau = 0
computeds0 = 0

liste_s0_sensitivity_max = 0
liste_s0_sensitivity_min = 0
liste_tau_sensitivity_max = 0
liste_tau_sensitivity_min = 0

eq_mid_generic = 0
factorization_integral_latex_equation_mid_generic = 0
factorization_1f1_latex_equation_mid_generic = 0
eq_mid_co = 0
eq_crrid_generic = 0

a_coeff_chain_mid_generic = ""
b_coeff_chain_mid_generic = ""
a_coeff_chain_mid_co = ""
b_coeff_chain_mid_co = ""
a_coeff_chain_crrid_generic = ""
b_coeff_chain_crrid_generic = ""

n_value_mid_generic = 0
m_value_mid_generic = 0
s0_value_mid_generic = 0
tau_value_mid_generic = 0

n_value_mid_co = 0
m_value_mid_co = 0
s0_value_mid_co = 0
user_choice_mid_co = ""
tau_value_mid_co = 0

n_value_crrid_generic = 0
m_value_crrid_generic = 0
s0_value_crrid_generic = 0
tau_value_crrid_generic = 0
d_value_crrid_generic = 0
    
# -----------------------------------------------------------------------------
# OUTPUT WIDGETS
# -----------------------------------------------------------------------------
# Generic MID tab

output_compute_analysis_mid_generic = widgets.Output()
output_time_simulation_computation_mid_generic = widgets.Output()

# -----------------------------------------------------------------------------
# Control oriented MID tab

output_admissibilite_computation_mid_co = widgets.Output()
output_compute_analysis_mid_co = widgets.Output()
output_sensibilite_computation_mid_co = widgets.Output()
output_time_simulation_computation_mid_co = widgets.Output()

# -----------------------------------------------------------------------------
# Generic CRRID tab

output_compute_analysis_crrid_generic = widgets.Output()
output_time_simulation_computation_crrid_generic = widgets.Output()

# -----------------------------------------------------------------------------
# PDF report tab

output_file_link_mid_generic = widgets.Output()
output_file_link_mid_co = widgets.Output()
output_file_link_crrid_generic = widgets.Output()

# -----------------------------------------------------------------------------
# Example tab

output_admissibilite_computation_example_1 = widgets.Output()
output_compute_analysis_example_1 = widgets.Output()


# -----------------------------------------------------------------------------
# USER INTERFACE INPUT
# -----------------------------------------------------------------------------
# Home tab


def slider_admin_user_mode_eventhandler(change):
    if change.new == "Admin":
        display(Javascript("""
        require(
            ["base/js/dialog"],
            function(dialog) {
                dialog.modal({
                    title: 'Information',
                    body: 'You are in the Administrator mode of P3Delta \
                    Online. The boundaries of the input values are no \
                    longer blocked. Caution! The waiting time of the \
                    calculations can be very long and the software unstable.',
                    buttons: {
                        'OK': {}
                    }
                });
            })
        """))
    if change.new == "User":
        display(Javascript("""
        require(
            ["base/js/dialog"],
            function(dialog) {
                dialog.modal({
                    title: 'Information',
                    body: 'You are in the User mode of P3Delta \
                    Online. The boundaries of the input values are \
                    limited.',
                    buttons: {
                        'OK': {}
                    }
                });
            })
        """))


slider_notification = widgets.SelectionSlider(
    options=['On', 'Off'],
    value='On',
    description='Notification : ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='200px'))

slider_admin_user_mode = widgets.SelectionSlider(
    options=['Admin', 'User'],
    value='User',
    description='Mode : ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='200px'))

slider_admin_user_mode.observe(slider_admin_user_mode_eventhandler,
                                 names='value')


# -----------------------------------------------------------------------------
# Generic MID tab


slider_n_mid_generic = widgets.IntSlider(min=1, max=7, step=1,
                                         description='n :', value=2)
slider_n_mid_generic.style.handle_color = '#ff9800'


def slider_n_mid_generic_eventhandler(change):
    slider_m_mid_generic.max = change.new
    if slider_notification.value == 'On' :
        if slider_n_mid_generic.max == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 101',
                    body: 'Maximum value of degree of polynomial n reached',
                    buttons: {
                        'OK': {}
                    }
            });})"""))
        if slider_n_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 103',
                    body: 'Minimum value of degree of polynomial n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


slider_n_mid_generic.observe(slider_n_mid_generic_eventhandler, names='value')

slider_m_mid_generic = widgets.IntSlider(min=0, max=2, step=1,
                                         description='m :', value=1)
slider_m_mid_generic.style.handle_color = '#ff9800'


def slider_m_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if slider_m_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 102',
                        body: 'Maximum value of degree of delay polynomial m \
                        reached. For sack of causality, degree of polynomial n \
                        should be higher \
                        or equal to degree of delay polynomial m',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if slider_m_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 104',
                    body: 'Minimum value of degree of delay polynomial m reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


slider_m_mid_generic.observe(slider_m_mid_generic_eventhandler, names='value')

floattext_s0_mid_generic = widgets.BoundedFloatText(min=-100.00, max=0.00,
                                                    step=0.01,
                                                    description='s0 :',
                                                    value=-5.00,
                                                    disabled=False)


def floattext_s0_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_s0_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 105',
                        body: 'Maximum value of rightmost root s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_s0_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 107',
                    body: 'Minimum value of rightmost root s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_s0_mid_generic.observe(floattext_s0_mid_generic_eventhandler,
                                 names='value')

floattext_tau_mid_generic = widgets.BoundedFloatText(min=0.00, max=1000.00,
                                                     step=0.01,
                                                     description='tau :',
                                                     value=1.00,
                                                     disabled=False)


def floattext_tau_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_tau_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 106',
                        body: 'Maximum value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_tau_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 108',
                    body: 'Minimum value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_tau_mid_generic.observe(floattext_tau_mid_generic_eventhandler,
                                  names='value')

button_ready_to_compute_mid_generic = widgets.Button(
    description='Compute analysis', layout=widgets.Layout(width='auto'),
    disabled=False, button_style='warning',
    icon='play-circle')
intprogress_1_mid_generic = widgets.IntProgress(
    layout=widgets.Layout(width='auto'), value=0, min=0, max=10,
    description='Loading  ', style={'bar_color': '#ff9800'},
    orientation='horizontal')
layout_validate_1_mid_generic = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_compute_mid_generic,
    center=None,
    right_sidebar=intprogress_1_mid_generic,
    footer=None)

constant_constant_mid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Constant :', disabled=True,
    min=-1000.00, max=1000.00)


def constant_constant_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if constant_constant_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 111',
                        body: 'Maximum value of initial condition constant c \
                        reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if constant_constant_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 113',
                    body: 'Minimum value of initial condition constant c \
                    reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


constant_constant_mid_generic.observe(
    constant_constant_mid_generic_eventhandler,
    names='value')

p = 9
items_floattext_polynomial_coef_mid_generic = [
    widgets.BoundedFloatText(step=0.01, description='a' + str(i), value=1.00,
                             min=-9999.99, max=9999.99,
                             disabled=False,
                             layout=widgets.Layout(max_width='800px')) for i in
    range(p - 1)]
box_0_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0]],
    layout=widgets.Layout(max_width='auto'))
box_1_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2],
     items_floattext_polynomial_coef_mid_generic[3]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2],
     items_floattext_polynomial_coef_mid_generic[3],
     items_floattext_polynomial_coef_mid_generic[4]],
    layout=widgets.Layout(max_width='auto'))
box_5_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2],
     items_floattext_polynomial_coef_mid_generic[3],
     items_floattext_polynomial_coef_mid_generic[4],
     items_floattext_polynomial_coef_mid_generic[5]],
    layout=widgets.Layout(max_width='auto'))
box_6_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2],
     items_floattext_polynomial_coef_mid_generic[3],
     items_floattext_polynomial_coef_mid_generic[4],
     items_floattext_polynomial_coef_mid_generic[5],
     items_floattext_polynomial_coef_mid_generic[6]],
    layout=widgets.Layout(max_width='auto'))
box_7_floattext_polynomial_coef_mid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_mid_generic[0],
     items_floattext_polynomial_coef_mid_generic[1],
     items_floattext_polynomial_coef_mid_generic[2],
     items_floattext_polynomial_coef_mid_generic[3],
     items_floattext_polynomial_coef_mid_generic[4],
     items_floattext_polynomial_coef_mid_generic[5],
     items_floattext_polynomial_coef_mid_generic[6],
     items_floattext_polynomial_coef_mid_generic[7]],
    layout=widgets.Layout(max_width='auto'))

stacked = widgets.Stacked([box_0_floattext_polynomial_coef_mid_generic,
                           box_1_floattext_polynomial_coef_mid_generic,
                           box_2_floattext_polynomial_coef_mid_generic,
                           box_3_floattext_polynomial_coef_mid_generic,
                           box_4_floattext_polynomial_coef_mid_generic,
                           box_5_floattext_polynomial_coef_mid_generic,
                           box_6_floattext_polynomial_coef_mid_generic,
                           box_7_floattext_polynomial_coef_mid_generic])
polynomial_degree_mid_generic = widgets.Dropdown(description='degree : ',
                                                 options=['0', '1', '2', '3',
                                                          '4', '5', '6', '7'])
widgets.jslink((polynomial_degree_mid_generic, 'index'),
               (stacked, 'selected_index'))
stacked_2_simulation_time_mid_generic = widgets.VBox(
    [polynomial_degree_mid_generic, stacked])
polynomial_mid_generic = widgets.HBox([stacked_2_simulation_time_mid_generic],
                                      layout=widgets.Layout(width='auto'))


def exponential_exponant_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_exponant_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 117',
                        body: 'Maximum value of exponant e reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_exponant_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 118',
                    body: 'Minimum value of exponant e reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_exponant_mid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Exponant :', disabled=False,
    min=0.00, max=100.00)

exponential_exponant_mid_generic.observe(
    exponential_exponant_mid_generic_eventhandler,
    names='value')


def exponential_gain_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_gain_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 119',
                        body: 'Maximum value of gain g reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_gain_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 120',
                    body: 'Minimum value of gain g reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_gain_mid_generic = widgets.BoundedFloatText(value=1.00, step=0.01,
                                                        min=-100.00,
                                                        max=100.00,
                                                        description='Gain :',
                                                        disabled=False)

exponential_gain_mid_generic.observe(
    exponential_gain_mid_generic_eventhandler,
    names='value')

exponential_mid_generic = widgets.HBox(
    [exponential_exponant_mid_generic, exponential_gain_mid_generic],
    layout=widgets.Layout(width='auto'))


def trigonometric_amplitude_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_amplitude_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 121',
                        body: 'Maximum value of amplitude a reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_amplitude_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 122',
                    body: 'Minimum value of amplitude a reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_frequency_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_frequency_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 123',
                        body: 'Maximum value of frequency f reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_frequency_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 124',
                    body: 'Minimum value of frequency f reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_phase_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_phase_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 125',
                        body: 'Maximum value of phase p reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_phase_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 126',
                    body: 'Minimum value of phase p reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


trigonometric_amplitude_mid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Amplitude :', disabled=False,
    min=-9999.99, max=9999.99)
trigonometric_frequency_mid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Frequency :', disabled=False,
    min=0.00, max=9999.99)
trigonometric_phase_mid_generic = widgets.BoundedFloatText(
    value=1.00,
    step=0.01,
    description='Phase :',
    disabled=False,
    min=-9999.99,
    max=9999.99)

trigonometric_amplitude_mid_generic.observe(
    trigonometric_amplitude_mid_generic_eventhandler,
    names='value')
trigonometric_frequency_mid_generic.observe(
    trigonometric_frequency_mid_generic_eventhandler,
    names='value')
trigonometric_phase_mid_generic.observe(
    trigonometric_phase_mid_generic_eventhandler,
    names='value')

trigonometric_mid_generic = widgets.HBox(
    [trigonometric_amplitude_mid_generic, trigonometric_frequency_mid_generic,
     trigonometric_phase_mid_generic], layout=widgets.Layout(width='auto'))

stacked_mid_generic = widgets.Stacked(
    [constant_constant_mid_generic, polynomial_mid_generic,
     exponential_mid_generic, trigonometric_mid_generic])
dropdown_mid_generic = widgets.Dropdown(
    options=['Constant', 'Polynomial', 'Exponential', 'Trigonometric'],
    disabled=True)

widgets.jslink((dropdown_mid_generic, 'index'),
               (stacked_mid_generic, 'selected_index'))

stacked_simulation_time_mid_generic = widgets.VBox(
    [dropdown_mid_generic, stacked_mid_generic])

floattext_simulation_time_mid_generic = widgets.BoundedFloatText(
    min=1.00,
    max=60.00,
    step=0.01,
    description='Simulation time :',
    value=10.00,
    disabled=True,
    style={
        'description_width': 'initial'})


def floattext_simulation_time_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_simulation_time_mid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 112',
                        body: 'Maximum value of time simulation t reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_simulation_time_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 114',
                    body: 'Minimum value of time simulation t reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_simulation_time_mid_generic.observe(
    floattext_simulation_time_mid_generic_eventhandler,
    names='value')

inttext_n_iter_mid_generic = widgets.BoundedIntText(
    value=10000,
    min=2,
    max=100000,
    step=1,
    description='n iterations :',
    disabled=True)


def inttext_n_iter_mid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if inttext_n_iter_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 115',
                        body: 'Maximum value of number of iteration n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if inttext_n_iter_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 116',
                    body: 'Minimum value of number of iteration n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


inttext_n_iter_mid_generic.observe(inttext_n_iter_mid_generic_eventhandler,
                                   names='value')

button_ready_to_simulation_time_mid_generic = widgets.Button(
    description='Compute time simulation', layout=widgets.Layout(width='auto'),
    disabled=True, button_style='danger', icon='play-circle')
intprogress_2_mid_generic = widgets.IntProgress(
    layout=widgets.Layout(width='auto'), value=0, min=0, max=10,
    description='Loading  ', style={'bar_color': '#f84434'},
    orientation='horizontal')
layout_validate_2_mid_generic = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_simulation_time_mid_generic,
    center=None,
    right_sidebar=intprogress_2_mid_generic,
    footer=None)


# -----------------------------------------------------------------------------
# Control oriented MID tab


def slider_n_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if change.new + 1 == 8:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 101',
                        body: 'Maximum value of degree of polynomial n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if change.new + 1 == 1:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 103',
                    body: 'Minimum value of degree of polynomial n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))
    slider_m_mid_co.max = change.new + 1
    for f in range(len(items_floattext_degree_n_coef_mid_co)):
        items_floattext_degree_n_coef_mid_co[f].disabled = False
    items_floattext_degree_n_coef_mid_co[change.new + 1].disabled = True


p = 10
items_floattext_degree_n_coef_mid_co = [
    widgets.BoundedFloatText(step=0.01, description='a' + str(i), value=1.00,
                             disabled=False,
                             min=-9999.99, max=9999.99,
                             layout=widgets.Layout(max_width='800px')) for i in
    range(p - 1)]

box_1_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3],
     items_floattext_degree_n_coef_mid_co[4]],
    layout=widgets.Layout(max_width='auto'))
box_5_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3],
     items_floattext_degree_n_coef_mid_co[4],
     items_floattext_degree_n_coef_mid_co[5]],
    layout=widgets.Layout(max_width='auto'))
box_6_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3],
     items_floattext_degree_n_coef_mid_co[4],
     items_floattext_degree_n_coef_mid_co[5],
     items_floattext_degree_n_coef_mid_co[6]],
    layout=widgets.Layout(max_width='auto'))
box_7_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3],
     items_floattext_degree_n_coef_mid_co[4],
     items_floattext_degree_n_coef_mid_co[5],
     items_floattext_degree_n_coef_mid_co[6],
     items_floattext_degree_n_coef_mid_co[7]],
    layout=widgets.Layout(max_width='auto'))
box_8_floattext_degree_n_coef_mid_co = widgets.VBox(
    [items_floattext_degree_n_coef_mid_co[0],
     items_floattext_degree_n_coef_mid_co[1],
     items_floattext_degree_n_coef_mid_co[2],
     items_floattext_degree_n_coef_mid_co[3],
     items_floattext_degree_n_coef_mid_co[4],
     items_floattext_degree_n_coef_mid_co[5],
     items_floattext_degree_n_coef_mid_co[6],
     items_floattext_degree_n_coef_mid_co[7],
     items_floattext_degree_n_coef_mid_co[8]],
    layout=widgets.Layout(max_width='auto'))

stacked_slider_n_mid_co = widgets.Stacked(
    [box_1_floattext_degree_n_coef_mid_co,
     box_2_floattext_degree_n_coef_mid_co,
     box_3_floattext_degree_n_coef_mid_co,
     box_4_floattext_degree_n_coef_mid_co,
     box_5_floattext_degree_n_coef_mid_co,
     box_6_floattext_degree_n_coef_mid_co,
     box_7_floattext_degree_n_coef_mid_co,
     box_8_floattext_degree_n_coef_mid_co])

slider_n_mid_co = widgets.SelectionSlider(
    options=[1, 2, 3, 4, 5, 6, 7, 8],
    value=2,
    description='n :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
slider_n_mid_co.style.handle_color = '#ff9800'

widgets.jslink((slider_n_mid_co, 'index'),
               (stacked_slider_n_mid_co, 'selected_index'))
vbox_stacked_slider_n_mid_co = widgets.VBox(
    [slider_n_mid_co, stacked_slider_n_mid_co])
degree_n_mid_co = widgets.HBox([vbox_stacked_slider_n_mid_co],
                               layout=widgets.Layout(width='auto'))

items_floattext_degree_n_coef_mid_co[slider_n_mid_co.value].disabled = True

slider_n_mid_co.observe(slider_n_mid_co_eventhandler, names='index')

slider_m_mid_co = widgets.IntSlider(min=0, max=slider_n_mid_co.value,
                                    step=1, description='m :', value=1)
slider_m_mid_co.style.handle_color = '#ff9800'


def slider_m_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if slider_m_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 102',
                        body: 'Maximum value of degree of delay polynomial m \
                        reached. Degree of polynomial n should be higher \
                        or equal to degree of delay polynomial m.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if slider_m_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 104',
                    body: 'Minimum value of degree of delay polynomial m reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


slider_m_mid_co.observe(slider_m_mid_co_eventhandler, names='value')

togglebuttons_limits_mid_co = widgets.ToggleButtons(
    options=['s0 and tau defaut limits', 's0 and tau custom limits'],
    description='',
    disabled=False,
    button_style='',
    tooltips=['The limits of s0 and tau are given by the software',
              'The limits of s0 and tau are given by the user'],
)
togglebuttons_limits_mid_co.style.button_width = 'auto'

boundeddloattext_s0_limits_min_mid_co = widgets.BoundedFloatText(
    value=-16.00, step=0.01, description='s0 min :', disabled=False,
    min=-50.00, max=0.00)
boundeddloattext_s0_limits_max_mid_co = widgets.BoundedFloatText(
    value=0.00, step=0.01, description='s0 max :', disabled=False,
    min=-50.00, max=50.00)
boundeddloattext_tau_limits_min_mid_co = widgets.BoundedFloatText(
    value=0.00, step=0.01, description='tau min :', disabled=False,
    min=0.00, max=10.00)
boundeddloattext_tau_limits_max_mid_co = widgets.BoundedFloatText(
    value=10.00, step=0.01, description='tau max :', disabled=False,
    min=0.00, max=50.00)


def boundeddloattext_s0_limits_min_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if boundeddloattext_s0_limits_min_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 127',
                        body: 'Maximum lower limit value of rightmost root \
                        s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_s0_limits_min_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 128',
                    body: 'Minimum lower limit value of rightmost root \
                    s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_s0_limits_max_mid_co_eventhandler(change):
    boundeddloattext_s0_limits_min_mid_co.max = \
        boundeddloattext_s0_limits_max_mid_co.value
    if slider_notification.value == 'On' :
        if boundeddloattext_s0_limits_max_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 129',
                        body: 'Maximum upper limit value of rightmost root \
                        s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_s0_limits_max_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 130',
                    body: 'Minimum upper limit value of rightmost root \
                    s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_tau_limits_min_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if boundeddloattext_tau_limits_min_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 131',
                        body: 'Maximum lower limit value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_tau_limits_min_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 132',
                    body: 'Minimum lower limit value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_tau_limits_max_mid_co_eventhandler(change):
    boundeddloattext_tau_limits_min_mid_co.max = \
        boundeddloattext_tau_limits_max_mid_co.value
    if slider_notification.value == 'On' :
        if boundeddloattext_tau_limits_max_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 133',
                        body: 'Maximum upper limit value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_tau_limits_max_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 134',
                    body: 'Minimum upper limit value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


boundeddloattext_s0_limits_min_mid_co.observe(
    boundeddloattext_s0_limits_min_mid_co_eventhandler,
    names='value')
boundeddloattext_s0_limits_max_mid_co.observe(
    boundeddloattext_s0_limits_max_mid_co_eventhandler,
    names='value')
boundeddloattext_tau_limits_min_mid_co.observe(
    boundeddloattext_tau_limits_min_mid_co_eventhandler,
    names='value')
boundeddloattext_tau_limits_max_mid_co.observe(
    boundeddloattext_tau_limits_max_mid_co_eventhandler,
    names='value')

hbox_1_limits_mid_co = widgets.HBox([boundeddloattext_s0_limits_min_mid_co,
                                     boundeddloattext_s0_limits_max_mid_co])
hbox_2_limits_mid_co = widgets.HBox([boundeddloattext_tau_limits_min_mid_co,
                                     boundeddloattext_tau_limits_max_mid_co])

vbox_1_limits_mid_co = widgets.VBox([])
vbox_2_limits_mid_co = widgets.VBox([
    hbox_1_limits_mid_co,
    hbox_2_limits_mid_co])

stacked_limits_mid_co = widgets.Stacked([
    vbox_1_limits_mid_co,
    vbox_2_limits_mid_co])

widgets.jslink((togglebuttons_limits_mid_co, 'index'),
               (stacked_limits_mid_co, 'selected_index'))

vbox_limits_mid_co = widgets.VBox([
    togglebuttons_limits_mid_co,
    stacked_limits_mid_co])

button_confirm_system_coeff_mid_co = widgets.Button(
    description='Confirm parameters', layout=widgets.Layout(width='49%'),
    disabled=False, button_style='warning', icon='angle-double-right')

togglebuttons_stable_limits_mid_co = widgets.ToggleButtons(
    options=['stable limits', 'arbitrary limits'],
    description='',
    disabled=True,
    button_style='',
    tooltips=['The limits of s0 and tau are set to have a stable system.',
              'The limits of s0 and tau are set arbitrarily.'])

togglebuttons_stable_limits_mid_co.style.button_width = 'auto'


def togglebuttons_stable_limits_mid_co_eventhandler(change):
    if change.new == 'stable limits':

        if togglebuttons_mid_co.value == 's0':
            if liste_s0_sensitivity_max <= floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.min = liste_s0_sensitivity_min
                floattext_s0_tau_mid_co.max = liste_s0_sensitivity_max
            elif liste_s0_sensitivity_max > floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.max = liste_s0_sensitivity_max
                floattext_s0_tau_mid_co.min = liste_s0_sensitivity_min

            floattext_s0_tau_mid_co.step = 0.001
            floattext_s0_tau_mid_co.value = -1.506
            floattext_s0_tau_mid_co.description = 's0 : '

        elif togglebuttons_mid_co.value == 'tau':
            if liste_s0_sensitivity_max <= floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min
                floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max
            elif liste_s0_sensitivity_max > floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max
                floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min

            floattext_s0_tau_mid_co.step = 0.01
            floattext_s0_tau_mid_co.value = 0.73
            floattext_s0_tau_mid_co.description = 'tau : '

    elif change.new == 'arbitrary limits':
        if togglebuttons_mid_co.value == 's0':
            floattext_s0_tau_mid_co.min = \
                boundeddloattext_s0_limits_min_mid_co.value
            floattext_s0_tau_mid_co.max = \
                boundeddloattext_s0_limits_max_mid_co.value

            floattext_s0_tau_mid_co.step = 0.001
            floattext_s0_tau_mid_co.value = -1.506
            floattext_s0_tau_mid_co.description = 's0 : '

        elif togglebuttons_mid_co.value == 'tau':
            floattext_s0_tau_mid_co.min = \
                boundeddloattext_tau_limits_min_mid_co.value
            floattext_s0_tau_mid_co.max = \
                boundeddloattext_tau_limits_max_mid_co.value

            floattext_s0_tau_mid_co.step = 0.01
            floattext_s0_tau_mid_co.value = 0.73
            floattext_s0_tau_mid_co.description = 'tau : '


togglebuttons_stable_limits_mid_co.observe(
    togglebuttons_stable_limits_mid_co_eventhandler, names='value')

togglebuttons_mid_co = widgets.ToggleButtons(options=['tau', 's0'],
                                             description='',
                                             disabled=True, button_style='',
                                             tooltips=['tau is known',
                                                       's0 is known'])

togglebuttons_mid_co.style.button_width = '150px'


def togglebuttons_mid_co_eventhandler(change):
    if change.new == 's0':

        if togglebuttons_stable_limits_mid_co.value == 'stable limits':
            if liste_s0_sensitivity_max <= floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.min = liste_s0_sensitivity_min
                floattext_s0_tau_mid_co.max = liste_s0_sensitivity_max
            elif liste_s0_sensitivity_max > floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.max = liste_s0_sensitivity_max
                floattext_s0_tau_mid_co.min = liste_s0_sensitivity_min

        elif togglebuttons_stable_limits_mid_co.value == 'arbitrary limits':
            floattext_s0_tau_mid_co.min = \
                boundeddloattext_s0_limits_min_mid_co.value
            floattext_s0_tau_mid_co.max = \
                boundeddloattext_s0_limits_max_mid_co.value

        floattext_s0_tau_mid_co.step = 0.001
        floattext_s0_tau_mid_co.value = -1.506
        floattext_s0_tau_mid_co.description = 's0 : '

    if change.new == 'tau':

        if togglebuttons_stable_limits_mid_co.value == 'stable limits':

            if liste_s0_sensitivity_max <= floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min
                floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max
            elif liste_s0_sensitivity_max > floattext_s0_tau_mid_co.min:
                floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max
                floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min

        elif togglebuttons_stable_limits_mid_co.value == 'arbitrary limits':
            floattext_s0_tau_mid_co.min = \
                boundeddloattext_tau_limits_min_mid_co.value
            floattext_s0_tau_mid_co.max = \
                boundeddloattext_tau_limits_max_mid_co.value

        floattext_s0_tau_mid_co.step = 0.01
        floattext_s0_tau_mid_co.value = 0.73
        floattext_s0_tau_mid_co.description = 'tau : '


togglebuttons_mid_co.observe(togglebuttons_mid_co_eventhandler, names='value')

floattext_s0_tau_mid_co = widgets.BoundedFloatText(description='tau : ',
                                                   disabled=True)

layout_validate_1_mid_co = widgets.AppLayout(
    header=None,
    left_sidebar=togglebuttons_stable_limits_mid_co,
    center=togglebuttons_mid_co,
    right_sidebar=floattext_s0_tau_mid_co,
    footer=None)

button_ready_to_compute_mid_co = widgets.Button(description='Compute analysis',
                                                layout=widgets.Layout(
                                                    width='auto'),
                                                disabled=True,
                                                button_style='warning',
                                                icon='play-circle')
intprogress_1_mid_co = widgets.IntProgress(layout=widgets.Layout(width='auto'),
                                           value=0, min=0, max=10,
                                           description='Loading  ',
                                           style={'bar_color': '#ff9c28'},
                                           orientation='horizontal')
layout_validate_2_mid_co = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_compute_mid_co,
    center=None,
    right_sidebar=intprogress_1_mid_co,
    footer=None)


inttext_n_iter_sensitivity_mid_co = widgets.BoundedIntText(
    value=10,
    min=2,
    max=100,
    step=1,
    description='n iterations :',
    disabled=True)

def inttext_n_iter_sensitivity_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if inttext_n_iter_sensitivity_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 115',
                        body: 'Maximum value of number of iteration n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if inttext_n_iter_sensitivity_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 116',
                    body: 'Minimum value of number of iteration n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


inttext_n_iter_sensitivity_mid_co.observe(inttext_n_iter_sensitivity_mid_co_eventhandler,
                                   names='value')

floattext_step_sensitivity_mid_co = widgets.BoundedFloatText(
    value=0.01,
    min=0.01,
    max=10,
    step=0.001,
    description='step :',
    disabled=True)

def floattext_step_sensitivity_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_step_sensitivity_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 137',
                        body: 'Maximum value of step reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_step_sensitivity_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 138',
                    body: 'Minimum value of step reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_step_sensitivity_mid_co.observe(floattext_step_sensitivity_mid_co_eventhandler,
                                   names='value')

layout_validate_5_mid_co = widgets.AppLayout(
    header=None,
    left_sidebar=inttext_n_iter_sensitivity_mid_co,
    center=None,
    right_sidebar=floattext_step_sensitivity_mid_co,
    footer=None)


button_ready_to_sensitivity_mid_co = widgets.Button(
    description='Compute sensitivity (optional)',
    layout=widgets.Layout(width='auto'), disabled=True, button_style='info',
    icon='play-circle')
intprogress_2_mid_co = widgets.IntProgress(layout=widgets.Layout(width='auto'),
                                           value=0, min=0, max=10,
                                           description='Loading  ',
                                           style={'bar_color': '#10acd8'},
                                           orientation='horizontal')
layout_validate_4_mid_co = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_sensitivity_mid_co,
    center=None,
    right_sidebar=intprogress_2_mid_co,
    footer=None)

constant_constant_mid_co = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Constant :', disabled=True,
    min=-1000.00, max=1000.00)


def constant_constant_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if constant_constant_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 111',
                        body: 'Maximum value of initial condition constant c \
                        reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if constant_constant_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 113',
                    body: 'Minimum value of initial condition constant c \
                    reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


constant_constant_mid_co.observe(constant_constant_mid_co_eventhandler,
                                 names='value')

p = 9
items_floattext_polynomial_coef_mid_co = [
    widgets.BoundedFloatText(step=0.01, description='a' + str(i), value=1.00,
                             min=-9999.99, max=9999.99,
                             disabled=False,
                             layout=widgets.Layout(max_width='800px')) for i in
    range(p - 1)]
box_0_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0]],
    layout=widgets.Layout(max_width='auto'))
box_1_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2],
     items_floattext_polynomial_coef_mid_co[3]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2],
     items_floattext_polynomial_coef_mid_co[3],
     items_floattext_polynomial_coef_mid_co[4]],
    layout=widgets.Layout(max_width='auto'))
box_5_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2],
     items_floattext_polynomial_coef_mid_co[3],
     items_floattext_polynomial_coef_mid_co[4],
     items_floattext_polynomial_coef_mid_co[5]],
    layout=widgets.Layout(max_width='auto'))
box_6_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2],
     items_floattext_polynomial_coef_mid_co[3],
     items_floattext_polynomial_coef_mid_co[4],
     items_floattext_polynomial_coef_mid_co[5],
     items_floattext_polynomial_coef_mid_co[6]],
    layout=widgets.Layout(max_width='auto'))
box_7_floattext_polynomial_coef_mid_co = widgets.VBox(
    [items_floattext_polynomial_coef_mid_co[0],
     items_floattext_polynomial_coef_mid_co[1],
     items_floattext_polynomial_coef_mid_co[2],
     items_floattext_polynomial_coef_mid_co[3],
     items_floattext_polynomial_coef_mid_co[4],
     items_floattext_polynomial_coef_mid_co[5],
     items_floattext_polynomial_coef_mid_co[6],
     items_floattext_polynomial_coef_mid_co[7]],
    layout=widgets.Layout(max_width='auto'))

stacked_mid_co = widgets.Stacked([box_0_floattext_polynomial_coef_mid_co,
                                  box_1_floattext_polynomial_coef_mid_co,
                                  box_2_floattext_polynomial_coef_mid_co,
                                  box_3_floattext_polynomial_coef_mid_co,
                                  box_4_floattext_polynomial_coef_mid_co,
                                  box_5_floattext_polynomial_coef_mid_co,
                                  box_6_floattext_polynomial_coef_mid_co,
                                  box_7_floattext_polynomial_coef_mid_co])
polynomial_degree_mid_co = widgets.Dropdown(description='degree : ',
                                            options=['0', '1', '2', '3', '4',
                                                     '5', '6', '7'])

widgets.jslink((polynomial_degree_mid_co, 'index'),
               (stacked_mid_co, 'selected_index'))
stacked_2_simulation_time_mid_co = widgets.VBox(
    [polynomial_degree_mid_co, stacked_mid_co])
polynomial_mid_co = widgets.HBox([stacked_2_simulation_time_mid_co],
                                 layout=widgets.Layout(width='auto'))


def exponential_exponant_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_exponant_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 117',
                        body: 'Maximum value of exponant e reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_exponant_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 118',
                    body: 'Minimum value of exponant e reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_exponant_mid_co = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Exponant :', disabled=False,
    min=0.00, max=100.00)

exponential_exponant_mid_co.observe(
    exponential_exponant_mid_co_eventhandler,
    names='value')


def exponential_gain_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_gain_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 119',
                        body: 'Maximum value of gain g reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_gain_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 120',
                    body: 'Minimum value of gain g reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_gain_mid_co = widgets.BoundedFloatText(value=1.00, step=0.01,
                                                   min=-100.00,
                                                   max=100.00,
                                                   description='Gain :',
                                                   disabled=False)

exponential_gain_mid_co.observe(
    exponential_gain_mid_co_eventhandler,
    names='value')

exponential_mid_co = widgets.HBox(
    [exponential_exponant_mid_co, exponential_gain_mid_co],
    layout=widgets.Layout(width='auto'))


def trigonometric_amplitude_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_amplitude_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 121',
                        body: 'Maximum value of amplitude a reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_amplitude_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 122',
                    body: 'Minimum value of amplitude a reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_frequency_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_frequency_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 123',
                        body: 'Maximum value of frequency f reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_frequency_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 124',
                    body: 'Minimum value of frequency f reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_phase_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_phase_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 125',
                        body: 'Maximum value of phase p reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_phase_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 126',
                    body: 'Minimum value of phase p reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


trigonometric_amplitude_mid_co = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Amplitude :', disabled=False,
    min=-9999.99, max=9999.99)
trigonometric_frequency_mid_co = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Frequency :', disabled=False,
    min=0.00, max=9999.99)
trigonometric_phase_mid_co = widgets.BoundedFloatText(value=1.00, step=0.01,
                                                      description='Phase :',
                                                      disabled=False,
                                                      min=-9999.99,
                                                      max=9999.99)

trigonometric_amplitude_mid_co.observe(
    trigonometric_amplitude_mid_co_eventhandler,
    names='value')
trigonometric_frequency_mid_co.observe(
    trigonometric_frequency_mid_co_eventhandler,
    names='value')
trigonometric_phase_mid_co.observe(
    trigonometric_phase_mid_co_eventhandler,
    names='value')

trigonometric_mid_co = widgets.HBox(
    [trigonometric_amplitude_mid_co, trigonometric_frequency_mid_co,
     trigonometric_phase_mid_co], layout=widgets.Layout(width='auto'))

stacked_mid_co = widgets.Stacked(
    [constant_constant_mid_co, polynomial_mid_co, exponential_mid_co,
     trigonometric_mid_co])
dropdown_mid_co = widgets.Dropdown(
    options=['Constant', 'Polynomial', 'Exponential', 'Trigonometric'],
    disabled=True)

widgets.jslink((dropdown_mid_co, 'index'), (stacked_mid_co, 'selected_index'))

stacked_simulation_time_mid_co = widgets.VBox(
    [dropdown_mid_co, stacked_mid_co])

floattext_simulation_time_mid_co = widgets.BoundedFloatText(
    min=1.00,
    max=60.00,
    step=0.01,
    description='Simulation time :',
    value=10.00,
    disabled=True,
    style={
        'description_width': 'initial'})


def floattext_simulation_time_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_simulation_time_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 112',
                        body: 'Maximum value of time simulation t reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_simulation_time_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 114',
                    body: 'Minimum value of time simulation t reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_simulation_time_mid_co.observe(
    floattext_simulation_time_mid_co_eventhandler,
    names='value')

inttext_n_iter_mid_co = widgets.BoundedIntText(
    value=10000,
    min=2,
    max=100000,
    step=1,
    description='n iterations :',
    disabled=True)


def inttext_n_iter_mid_co_eventhandler(change):
    if slider_notification.value == 'On' :
        if inttext_n_iter_mid_co.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 115',
                        body: 'Maximum value of number of iteration n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if inttext_n_iter_mid_co.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 116',
                    body: 'Minimum value of number of iteration n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


inttext_n_iter_mid_co.observe(inttext_n_iter_mid_co_eventhandler,
                              names='value')

button_ready_to_simulation_time_mid_co = widgets.Button(
    description='Compute time simulation', layout=widgets.Layout(width='auto'),
    disabled=True, button_style='danger', icon='play-circle')
intprogress_3_mid_co = widgets.IntProgress(layout=widgets.Layout(width='auto'),
                                           value=0, min=0, max=10,
                                           description='Loading  ',
                                           style={'bar_color': '#f84434'},
                                           orientation='horizontal')
layout_validate_3_mid_co = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_simulation_time_mid_co,
    center=None,
    right_sidebar=intprogress_3_mid_co,
    footer=None)

# -----------------------------------------------------------------------------
# Generic CRRID tab


slider_n_crrid_generic = widgets.IntSlider(min=1, max=7, step=1,
                                           description='n :', value=2)
slider_n_crrid_generic.style.handle_color = '#ff9800'


def slider_n_crrid_generic_eventhandler(change):
    slider_m_crrid_generic.max = change.new
    if slider_notification.value == 'On' :
        if slider_n_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 101',
                        body: 'Maximum value of degree of polynomial n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if slider_n_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 103',
                    body: 'Minimum value of degree of polynomial n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


slider_n_crrid_generic.observe(slider_n_crrid_generic_eventhandler,
                               names='value')
slider_m_crrid_generic = widgets.IntSlider(min=0, max=2, step=1,
                                           description='m :', value=1)
slider_m_crrid_generic.style.handle_color = '#ff9800'


def slider_m_crrid_eventhandler(change):
    if slider_notification.value == 'On' :
        if slider_m_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 102',
                        body: 'Maximum value of degree of delay polynomial m \
                        reached. Degree of polynomial n should be higher \
                        or equal to degree of delay polynomial m.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if slider_m_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 104',
                    body: 'Minimum value of degree of delay polynomial m reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


slider_m_crrid_generic.observe(slider_m_crrid_eventhandler, names='value')

floattext_d_crrid_generic = widgets.BoundedFloatText(value=1.00, min=0.00,
                                                     max=50.00,
                                                     step=0.01,
                                                     description='d :',
                                                     disabled=False)


def floattext_d_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_d_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 109',
                        body: 'Maximum value of step d reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_d_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 110',
                    body: 'Minimum value of step d reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_d_crrid_generic.observe(floattext_d_crrid_generic_eventhandler,
                                  names='value')

floattext_s0_crrid_generic = widgets.BoundedFloatText(min=-100.00, max=0.00,
                                                      step=0.01,
                                                      description='s0 :',
                                                      value=-1.00,
                                                      disabled=False)


def floattext_s0_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_s0_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 105',
                        body: 'Maximum value of rightmost root s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_s0_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 107',
                    body: 'Minimum value of rightmost root s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_s0_crrid_generic.observe(floattext_s0_crrid_generic_eventhandler,
                                   names='value')

floattext_tau_crrid_generic = widgets.BoundedFloatText(min=0.00, max=1000.00,
                                                       step=0.01,
                                                       description='tau :',
                                                       value=1.00,
                                                       disabled=False)


def floattext_tau_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_tau_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 106',
                        body: 'Maximum value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_tau_mid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 108',
                    body: 'Minimum value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_tau_crrid_generic.observe(floattext_tau_crrid_generic_eventhandler,
                                    names='value')

button_ready_to_compute_crrid_generic = widgets.Button(
    description='Compute analysis', layout=widgets.Layout(width='auto'),
    disabled=False, button_style='warning',
    icon='play-circle')
intprogress_1_crrid_generic = widgets.IntProgress(
    layout=widgets.Layout(width='auto'), value=0, min=0, max=10,
    description='Loading  ', style={'bar_color': '#ff9800'},
    orientation='horizontal')
layout_validate_1_crrid_generic = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_compute_crrid_generic,
    center=None,
    right_sidebar=intprogress_1_crrid_generic,
    footer=None)

constant_constant_crrid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Constant :', disabled=True,
    min=-1000.00, max=1000.00)


def constant_constant_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if constant_constant_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 111',
                        body: 'Maximum value of initial condition constant c \
                        reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if constant_constant_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 113',
                    body: 'Minimum value of initial condition constant c \
                    reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


constant_constant_crrid_generic.observe(
    constant_constant_crrid_generic_eventhandler,
    names='value')

p = 9
items_floattext_polynomial_coef_crrid_generic = [
    widgets.BoundedFloatText(step=0.01, description='a' + str(i), value=1.00,
                             min=-9999.99, max=9999.99,
                             disabled=False,
                             layout=widgets.Layout(max_width='800px')) for i in
    range(p - 1)]
box_0_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0]],
    layout=widgets.Layout(max_width='auto'))
box_1_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2],
     items_floattext_polynomial_coef_crrid_generic[3]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2],
     items_floattext_polynomial_coef_crrid_generic[3],
     items_floattext_polynomial_coef_crrid_generic[4]],
    layout=widgets.Layout(max_width='auto'))
box_5_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2],
     items_floattext_polynomial_coef_crrid_generic[3],
     items_floattext_polynomial_coef_crrid_generic[4],
     items_floattext_polynomial_coef_crrid_generic[5]],
    layout=widgets.Layout(max_width='auto'))
box_6_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2],
     items_floattext_polynomial_coef_crrid_generic[3],
     items_floattext_polynomial_coef_crrid_generic[4],
     items_floattext_polynomial_coef_crrid_generic[5],
     items_floattext_polynomial_coef_crrid_generic[6]],
    layout=widgets.Layout(max_width='auto'))
box_7_floattext_polynomial_coef_crrid_generic = widgets.VBox(
    [items_floattext_polynomial_coef_crrid_generic[0],
     items_floattext_polynomial_coef_crrid_generic[1],
     items_floattext_polynomial_coef_crrid_generic[2],
     items_floattext_polynomial_coef_crrid_generic[3],
     items_floattext_polynomial_coef_crrid_generic[4],
     items_floattext_polynomial_coef_crrid_generic[5],
     items_floattext_polynomial_coef_crrid_generic[6],
     items_floattext_polynomial_coef_crrid_generic[7]],
    layout=widgets.Layout(max_width='auto'))

stacked = widgets.Stacked([box_0_floattext_polynomial_coef_crrid_generic,
                           box_1_floattext_polynomial_coef_crrid_generic,
                           box_2_floattext_polynomial_coef_crrid_generic,
                           box_3_floattext_polynomial_coef_crrid_generic,
                           box_4_floattext_polynomial_coef_crrid_generic,
                           box_5_floattext_polynomial_coef_crrid_generic,
                           box_6_floattext_polynomial_coef_crrid_generic,
                           box_7_floattext_polynomial_coef_crrid_generic])
polynomial_degree_crrid_generic = widgets.Dropdown(description='degree : ',
                                                   options=['0', '1', '2', '3',
                                                            '4', '5', '6',
                                                            '7'])
widgets.jslink((polynomial_degree_crrid_generic, 'index'),
               (stacked, 'selected_index'))
stacked_2_simulation_time_crrid_generic = widgets.VBox(
    [polynomial_degree_crrid_generic, stacked])
polynomial_crrid_generic = widgets.HBox(
    [stacked_2_simulation_time_crrid_generic],
    layout=widgets.Layout(width='auto'))


def exponential_exponant_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_exponant_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 117',
                        body: 'Maximum value of exponant e reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_exponant_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 118',
                    body: 'Minimum value of exponant e reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_exponant_crrid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Exponant :', disabled=False,
    min=0.00, max=100.00)

exponential_exponant_crrid_generic.observe(
    exponential_exponant_crrid_generic_eventhandler,
    names='value')


def exponential_gain_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if exponential_gain_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 119',
                        body: 'Maximum value of gain g reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if exponential_gain_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 120',
                    body: 'Minimum value of gain g reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


exponential_gain_crrid_generic = widgets.BoundedFloatText(value=1.00,
                                                          step=0.01,
                                                          min=-100.00,
                                                          max=100.00,
                                                          description='Gain :',
                                                          disabled=False)

exponential_gain_crrid_generic.observe(
    exponential_gain_crrid_generic_eventhandler,
    names='value')

exponential_crrid_generic = widgets.HBox(
    [exponential_exponant_crrid_generic, exponential_gain_crrid_generic],
    layout=widgets.Layout(width='auto'))


def trigonometric_amplitude_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_amplitude_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 121',
                        body: 'Maximum value of amplitude a reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_amplitude_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 122',
                    body: 'Minimum value of amplitude a reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_frequency_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_frequency_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 123',
                        body: 'Maximum value of frequency f reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_frequency_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 124',
                    body: 'Minimum value of frequency f reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def trigonometric_phase_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if trigonometric_phase_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 125',
                        body: 'Maximum value of phase p reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if trigonometric_phase_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 126',
                    body: 'Minimum value of phase p reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


trigonometric_amplitude_crrid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Amplitude :', disabled=False,
    min=-9999.99, max=9999.99)
trigonometric_frequency_crrid_generic = widgets.BoundedFloatText(
    value=1.00, step=0.01, description='Frequency :', disabled=False,
    min=0.00, max=9999.99)
trigonometric_phase_crrid_generic = widgets.BoundedFloatText(
    value=1.00,
    step=0.01,
    description='Phase :',
    disabled=False,
    min=-9999.99,
    max=9999.99)

trigonometric_amplitude_crrid_generic.observe(
    trigonometric_amplitude_crrid_generic_eventhandler,
    names='value')
trigonometric_frequency_crrid_generic.observe(
    trigonometric_frequency_crrid_generic_eventhandler,
    names='value')
trigonometric_phase_crrid_generic.observe(
    trigonometric_phase_crrid_generic_eventhandler,
    names='value')

trigonometric_crrid_generic = widgets.HBox(
    [trigonometric_amplitude_crrid_generic,
     trigonometric_frequency_crrid_generic, trigonometric_phase_crrid_generic],
    layout=widgets.Layout(width='auto'))

stacked_crrid_generic = widgets.Stacked(
    [constant_constant_crrid_generic, polynomial_crrid_generic,
     exponential_crrid_generic, trigonometric_crrid_generic])
dropdown_crrid_generic = widgets.Dropdown(
    options=['Constant', 'Polynomial', 'Exponential', 'Trigonometric'],
    disabled=True)

widgets.jslink((dropdown_crrid_generic, 'index'),
               (stacked_crrid_generic, 'selected_index'))

stacked_simulation_time_crrid_generic = widgets.VBox(
    [dropdown_crrid_generic, stacked_crrid_generic])

floattext_simulation_time_crrid_generic = widgets.BoundedFloatText(
    min=1.00,
    max=60.00,
    step=0.01,
    description='Simulation time :',
    value=10.00,
    disabled=True,
    style={
        'description_width': 'initial'})


def floattext_simulation_time_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if floattext_simulation_time_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 112',
                        body: 'Maximum value of time simulation t reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if floattext_simulation_time_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 114',
                    body: 'Minimum value of time simulation t reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


floattext_simulation_time_crrid_generic.observe(
    floattext_simulation_time_crrid_generic_eventhandler,
    names='value')

inttext_n_iter_crrid_generic = widgets.BoundedIntText(
    value=10000,
    min=2,
    max=100000,
    step=1,
    description='n iterations :',
    disabled=True)


def inttext_n_iter_crrid_generic_eventhandler(change):
    if slider_notification.value == 'On' :
        if inttext_n_iter_crrid_generic.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 115',
                        body: 'Maximum value of number of iteration n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if inttext_n_iter_crrid_generic.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 116',
                    body: 'Minimum value of number of iteration n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


inttext_n_iter_crrid_generic.observe(inttext_n_iter_crrid_generic_eventhandler,
                                     names='value')

button_ready_to_simulation_time_crrid_generic = widgets.Button(
    description='Compute time simulation', layout=widgets.Layout(width='auto'),
    disabled=True, button_style='danger', icon='play-circle')
intprogress_2_crrid_generic = widgets.IntProgress(
    layout=widgets.Layout(width='auto'), value=0, min=0, max=10,
    description='Loading  ', style={'bar_color': '#f84434'},
    orientation='horizontal')
layout_validate_2_crrid_generic = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_simulation_time_crrid_generic,
    center=None,
    right_sidebar=intprogress_2_crrid_generic,
    footer=None)

# -----------------------------------------------------------------------------
# Example tab

def slider_n_example_1_eventhandler(change):
    if slider_notification.value == 'On' :
        if change.new + 1 == 4:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 135',
                        body: 'Maximum value of number of modes of the reduced model n reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if change.new + 1 == 1:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 136',
                    body: 'Minimum value of number of modes of the reduced model n reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))
    #for f in range(len(items_floattext_degree_n_coef_example_1_2)):
    #    items_floattext_degree_n_coef_example_1_2[f].disabled = False
    #items_floattext_degree_n_coef_example_1_5[2 * change.new + 2].disabled = True


p = 10
items_floattext_degree_n_coef_example_1_1 = [
    widgets.BoundedFloatText(step=0.01, description='(Nuy) ' + str(i), value=1.00,
                             disabled=False,
                             min=-99999.99, max=99999.99,
                             layout=widgets.Layout(max_width='175px')) for i in
    range(p - 1)]
items_floattext_degree_n_coef_example_1_2 = [
    widgets.BoundedFloatText(step=0.01, description='(psi) ' + str(i), value=1.00,
                             disabled=False,
                             min=-99999.99, max=99999.99,
                             layout=widgets.Layout(max_width='175px')) for i in
    range(p - 1)]

# Valeurs pour le PMI par défaut
items_floattext_degree_n_coef_example_1_1[0].value = 0.0824705565013658
items_floattext_degree_n_coef_example_1_1[1].value = 0.0402320642368774
items_floattext_degree_n_coef_example_1_1[2].value = 5472.41008648971

items_floattext_degree_n_coef_example_1_2[0].value = 1
items_floattext_degree_n_coef_example_1_2[1].value = 0.487835488732404
items_floattext_degree_n_coef_example_1_2[2].value = 59495.8660165543

items_floattext_degree_n_coef_example_1_2[0].disabled = True
    

box_1_floattext_degree_n_coef_example_1_1 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_1[0],
     items_floattext_degree_n_coef_example_1_1[1],
     items_floattext_degree_n_coef_example_1_1[2]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_degree_n_coef_example_1_1 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_1[0],
     items_floattext_degree_n_coef_example_1_1[1],
     items_floattext_degree_n_coef_example_1_1[2],
     items_floattext_degree_n_coef_example_1_1[3],
     items_floattext_degree_n_coef_example_1_1[4]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_degree_n_coef_example_1_1 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_1[0],
     items_floattext_degree_n_coef_example_1_1[1],
     items_floattext_degree_n_coef_example_1_1[2],
     items_floattext_degree_n_coef_example_1_1[3],
     items_floattext_degree_n_coef_example_1_1[4],
     items_floattext_degree_n_coef_example_1_1[5],
     items_floattext_degree_n_coef_example_1_1[6]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_degree_n_coef_example_1_1 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_1[0],
     items_floattext_degree_n_coef_example_1_1[1],
     items_floattext_degree_n_coef_example_1_1[2],
     items_floattext_degree_n_coef_example_1_1[3],
     items_floattext_degree_n_coef_example_1_1[4],
     items_floattext_degree_n_coef_example_1_1[5],
     items_floattext_degree_n_coef_example_1_1[6],
     items_floattext_degree_n_coef_example_1_1[7],
     items_floattext_degree_n_coef_example_1_1[8]],
    layout=widgets.Layout(max_width='auto'))

box_1_floattext_degree_n_coef_example_1_2 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_2[0],
     items_floattext_degree_n_coef_example_1_2[1],
     items_floattext_degree_n_coef_example_1_2[2]],
    layout=widgets.Layout(max_width='auto'))
box_2_floattext_degree_n_coef_example_1_2 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_2[0],
     items_floattext_degree_n_coef_example_1_2[1],
     items_floattext_degree_n_coef_example_1_2[2],
     items_floattext_degree_n_coef_example_1_2[3],
     items_floattext_degree_n_coef_example_1_2[4]],
    layout=widgets.Layout(max_width='auto'))
box_3_floattext_degree_n_coef_example_1_2 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_2[0],
     items_floattext_degree_n_coef_example_1_2[1],
     items_floattext_degree_n_coef_example_1_2[2],
     items_floattext_degree_n_coef_example_1_2[3],
     items_floattext_degree_n_coef_example_1_2[4],
     items_floattext_degree_n_coef_example_1_2[5],
     items_floattext_degree_n_coef_example_1_2[6]],
    layout=widgets.Layout(max_width='auto'))
box_4_floattext_degree_n_coef_example_1_2 = widgets.VBox(
    [items_floattext_degree_n_coef_example_1_2[0],
     items_floattext_degree_n_coef_example_1_2[1],
     items_floattext_degree_n_coef_example_1_2[2],
     items_floattext_degree_n_coef_example_1_2[3],
     items_floattext_degree_n_coef_example_1_2[4],
     items_floattext_degree_n_coef_example_1_2[5],
     items_floattext_degree_n_coef_example_1_2[6],
     items_floattext_degree_n_coef_example_1_2[7],
     items_floattext_degree_n_coef_example_1_2[8]],
    layout=widgets.Layout(max_width='auto'))


box_1_floattext_degree_n_coef_example_1_total = widgets.HBox(
    [box_1_floattext_degree_n_coef_example_1_1,
     box_1_floattext_degree_n_coef_example_1_2])
box_2_floattext_degree_n_coef_example_1_total = widgets.HBox(
    [box_2_floattext_degree_n_coef_example_1_1,
     box_2_floattext_degree_n_coef_example_1_2])
box_3_floattext_degree_n_coef_example_1_total = widgets.HBox(
    [box_3_floattext_degree_n_coef_example_1_1,
     box_3_floattext_degree_n_coef_example_1_2])
box_4_floattext_degree_n_coef_example_1_total = widgets.HBox(
    [box_4_floattext_degree_n_coef_example_1_1,
     box_4_floattext_degree_n_coef_example_1_2])

stacked_slider_n_example_1 = widgets.Stacked(
    [box_1_floattext_degree_n_coef_example_1_total,
     box_2_floattext_degree_n_coef_example_1_total,
     box_3_floattext_degree_n_coef_example_1_total,
     box_4_floattext_degree_n_coef_example_1_total])

slider_n_example_1 = widgets.SelectionSlider(
    options=[1, 2, 3, 4],
    value=1,
    description='n :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)
slider_n_example_1.style.handle_color = '#ff9800'

widgets.jslink((slider_n_example_1, 'index'),
               (stacked_slider_n_example_1, 'selected_index'))
vbox_stacked_slider_n_example_1 = widgets.VBox(
    [slider_n_example_1, stacked_slider_n_example_1])
degree_n_example_1 = widgets.HBox([vbox_stacked_slider_n_example_1],
                               layout=widgets.Layout(width='auto'))

#items_floattext_degree_n_coef_example_1_2[2 * slider_n_example_1.value].disabled = True

slider_n_example_1.observe(slider_n_example_1_eventhandler, names='index')




boundeddloattext_s0_limits_min_example_1 = widgets.BoundedFloatText(
    value=-10.00, step=0.01, description='s0 min :', disabled=False,
    min=-50.00, max=0.00)
boundeddloattext_s0_limits_max_example_1 = widgets.BoundedFloatText(
    value=0.00, step=0.01, description='s0 max :', disabled=False,
    min=-50.00, max=50.00)
boundeddloattext_tau_limits_min_example_1 = widgets.BoundedFloatText(
    value=0.00, step=0.01, description='tau min :', disabled=False,
    min=0.00, max=10.00)
boundeddloattext_tau_limits_max_example_1 = widgets.BoundedFloatText(
    value=10.00, step=0.01, description='tau max :', disabled=False,
    min=0.00, max=50.00)


def boundeddloattext_s0_limits_min_example_1_eventhandler(change):
    if slider_notification.value == 'On' :
        if boundeddloattext_s0_limits_min_example_1.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 127',
                        body: 'Maximum lower limit value of rightmost root \
                        s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_s0_limits_min_example_1.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 128',
                    body: 'Minimum lower limit value of rightmost root \
                    s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_s0_limits_max_example_1_eventhandler(change):
    boundeddloattext_s0_limits_min_example_1.max = \
        boundeddloattext_s0_limits_max_example_1.value
    if slider_notification.value == 'On' :
        if boundeddloattext_s0_limits_max_example_1.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 129',
                        body: 'Maximum upper limit value of rightmost root \
                        s0 reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_s0_limits_max_example_1.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 130',
                    body: 'Minimum upper limit value of rightmost root \
                    s0 reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_tau_limits_min_example_1_eventhandler(change):
    if slider_notification.value == 'On' :
        if boundeddloattext_tau_limits_min_example_1.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 131',
                        body: 'Maximum lower limit value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_tau_limits_min_example_1.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 132',
                    body: 'Minimum lower limit value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


def boundeddloattext_tau_limits_max_example_1_eventhandler(change):
    boundeddloattext_tau_limits_min_example_1.max = \
        boundeddloattext_tau_limits_max_example_1.value
    if slider_notification.value == 'On' :
        if boundeddloattext_tau_limits_max_example_1.max == change.new:
            display(Javascript("""
            require(
                ["base/js/dialog"],
                function(dialog) {
                    dialog.modal({
                        title: 'Error 133',
                        body: 'Maximum upper limit value of delay tau reached.',
                        buttons: {
                            'OK': {}
                        }
                    });
                })
            """))
        if boundeddloattext_tau_limits_max_example_1.min == change.new:
            display(Javascript("""require(["base/js/dialog"],function(dialog) {
                dialog.modal({
                    title: 'Error 134',
                    body: 'Minimum upper limit value of delay tau reached.',
                    buttons: {
                        'OK': {}
                    }
            });})"""))


boundeddloattext_s0_limits_min_example_1.observe(
    boundeddloattext_s0_limits_min_example_1_eventhandler,
    names='value')
boundeddloattext_s0_limits_max_example_1.observe(
    boundeddloattext_s0_limits_max_example_1_eventhandler,
    names='value')
boundeddloattext_tau_limits_min_example_1.observe(
    boundeddloattext_tau_limits_min_example_1_eventhandler,
    names='value')
boundeddloattext_tau_limits_max_example_1.observe(
    boundeddloattext_tau_limits_max_example_1_eventhandler,
    names='value')

hbox_1_limits_example_1 = widgets.HBox([boundeddloattext_s0_limits_min_example_1,
                                     boundeddloattext_s0_limits_max_example_1])
hbox_2_limits_example_1 = widgets.HBox([boundeddloattext_tau_limits_min_example_1,
                                     boundeddloattext_tau_limits_max_example_1])

vbox_1_limits_example_1 = widgets.VBox([
    hbox_1_limits_example_1,
    hbox_2_limits_example_1])

def computation_1_example_1():

    global liste_s0_sensitivity_max
    global liste_s0_sensitivity_min
    global liste_tau_sensitivity_max
    global liste_tau_sensitivity_min

    output_admissibilite_computation_example_1.clear_output()
    output_compute_analysis_example_1.clear_output()
    
    floattext_s0_example_1.disabled = True
    button_ready_to_compute_example_1.disabled = True
    intprogress_1_example_1.value = 0
    

    with output_admissibilite_computation_example_1:
        
        deg = 2*slider_n_example_1.value
        s, tau, n_0, nr_0, d_0, dr_0, nuy_sym, a_sym = define_symbols(deg)
        
        nuy_num = []
        for i in range((slider_n_example_1.value)*2 + 1):
            nuy_num.append(items_floattext_degree_n_coef_example_1_1[i].value)
        nuy_num = np.flip(np.array(nuy_num))
        
        a_num = []
        for i in range((slider_n_example_1.value)*2 + 1):
            a_num.append(items_floattext_degree_n_coef_example_1_2[i].value)
        a_num = np.flip(np.array(a_num))
        
        #print("ZONE DE TEST")
        #print("nuy_num",nuy_num)
        #print("a_num",a_num)
        
        delta_poly = contruct_quasipolynomial(deg, s, n_0, nr_0, d_0, dr_0, tau, a_sym, nuy_sym)
        n_0_sol, nr_0_sol, delta_poly = solution_n0_nr_0_and_subs(delta_poly, s, tau, n_0, nr_0, deg)
        d_0_sol, dr_0_sol = solve_d0_dr_0(delta_poly, s, d_0, dr_0)
        poly_admissibility = construct_admissibility_poly(delta_poly, s, tau, d_0_sol, dr_0_sol, d_0, dr_0)
        poly_admissibility = subs_parameters(poly_admissibility, nuy_num, nuy_sym, a_num, a_sym)

        
        s0range = np.linspace(boundeddloattext_s0_limits_min_example_1.value,
                              boundeddloattext_s0_limits_max_example_1.value,
                              num=1000)
        taurange = np.linspace(
            boundeddloattext_tau_limits_min_example_1.value,
            boundeddloattext_tau_limits_max_example_1.value,
            num=1000)

        func = sp.lambdify([tau, s], poly_admissibility)

        display(Markdown("<h3>Admissibility plot</h3>"))

        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        x, y = np.meshgrid(s0range, taurange)
        z = func(y, x)

        cs = ax.contour(x, y, z, [0], colors='#ff9c28')
        ax.grid()

        plt.xlabel(r"$s_0$")
        plt.ylabel(r"$\tau$")
        plt.title("(P3delta) Admissibility plot (example 1)")
        plt.savefig('results/admissibility_plot_example_1.png')

        p = cs.collections[0].get_paths()[0]
        v = p.vertices
        liste_s0_sensitivity = v[:, 0].copy()
        liste_tau_sensitivity = v[:, 1].copy()

        liste_s0_sensitivity_max = round(np.max(liste_s0_sensitivity), 5)
        liste_s0_sensitivity_min = round(np.min(liste_s0_sensitivity), 5)
        liste_tau_sensitivity_max = round(np.max(liste_tau_sensitivity), 5)
        liste_tau_sensitivity_min = round(np.min(liste_tau_sensitivity), 5)

        display(Markdown("<h3>s0 and tau stable system limits</h3>"))
        display(Markdown(
            r"\$" + str(liste_s0_sensitivity_min) + "\\leq s_0 \\leq " + str(
                liste_s0_sensitivity_max) + "$"))
        display(Markdown(
            r"\$" + str(liste_tau_sensitivity_min) + "\\leq tau \\leq " + str(
                liste_tau_sensitivity_max) + "$"))



def button_confirm_system_coeff_clicked_example_1(self):

    #input_interface_reset_mid_co()

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Coefficients of example 1 analysis confirmed !',
                buttons: {
                    'OK': {}
                }
        });})"""))
    computation_1_example_1()
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Admissibility compute done ! Example 1 analysis \
                is available.',
                buttons: {
                    'OK': {}
                }
        });})"""))
                
    floattext_s0_example_1.disabled = False
    button_ready_to_compute_example_1.disabled = False


button_confirm_system_coeff_example_1 = widgets.Button(
    description='Confirm parameters', layout=widgets.Layout(width='49%'),
    disabled=False, button_style='warning', icon='angle-double-right')

button_confirm_system_coeff_example_1.on_click(
    button_confirm_system_coeff_clicked_example_1)

floattext_s0_example_1 = widgets.BoundedFloatText(min=-1000.00, max=0.00,
                                                    step=0.01,
                                                    description='s0 :',
                                                    value=-10.00,
                                                    disabled=True)


def computation_2_example_1():
    
    output_compute_analysis_example_1.clear_output()
    
    with output_compute_analysis_example_1:
        
        intprogress_1_example_1.value = 6
        
        display(Markdown("<h3>Spectrum distribution</h3>"))
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        ax.set_title("(P3delta) Spectrum distribution (example 1)")
        ax.set_xlabel(r"$\Re(s)$")
        ax.set_ylabel(r"$\Im(s)$")
        ax.axhline(linewidth=2, color='black', zorder=2)
        ax.axvline(linewidth=2, color='black', zorder=2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder=2,
                   label='Unstable system')
        ax.grid()

        line, = ax.plot([], [], 'o', color='steelblue', zorder=3,
                        label='Roots')
        
        deg = 2*slider_n_example_1.value
        s, tau, n_0, nr_0, d_0, dr_0, nuy_sym, a_sym = define_symbols(deg)
        
        nuy_num = []
        for i in range((slider_n_example_1.value)*2 + 1):
            nuy_num.append(items_floattext_degree_n_coef_example_1_1[i].value)
        nuy_num = np.flip(np.array(nuy_num))
        
        a_num = []
        for i in range((slider_n_example_1.value)*2 + 1):
            a_num.append(items_floattext_degree_n_coef_example_1_2[i].value)
        a_num = np.flip(np.array(a_num))
        
        delta_poly = contruct_quasipolynomial(deg, s, n_0, nr_0, d_0, dr_0, tau, a_sym, nuy_sym)
        n_0_sol, nr_0_sol, delta_poly = solution_n0_nr_0_and_subs(delta_poly, s, tau, n_0, nr_0, deg)
        d_0_sol, dr_0_sol = solve_d0_dr_0(delta_poly, s, d_0, dr_0)
        poly_admissibility = construct_admissibility_poly(delta_poly, s, tau, d_0_sol, dr_0_sol, d_0, dr_0)
        poly_admissibility = subs_parameters(poly_admissibility, nuy_num, nuy_sym, a_num, a_sym)

        s0 = floattext_s0_example_1.value
        
        num_tau = solve_tau(poly_admissibility, s, tau, s0)
        num_n_0, num_nr_0, num_d_0, num_dr_0, num_tau = compute_and_display_gains(num_tau, d_0_sol, dr_0_sol,
                                                                         n_0_sol, nr_0_sol, s0, a_sym,
                                                                         a_num, nuy_sym, nuy_num, s, tau, d_0, dr_0)
        delta_poly = get_final_quasipolynomial(delta_poly, s, tau, num_d_0, num_dr_0, num_tau,
                                               a_sym, a_num, nuy_sym, nuy_num, d_0, dr_0)

        xr, yr, roots = find_roots(delta_poly, s)
        
        
        xmax = max(xr)
        plt.autoscale()
        line.set_data(xr, yr)
        xmax_round = round(xmax, 2)
        line2, = ax.plot([], [], '*', markersize=13, color='#ff9900', zorder=3,
                         label='Rightmost root (' + str(
                             xmax_round) + ';' + '0' + ')')
        line2.set_data(xmax, 0)
        ax.legend(loc='upper right')
        ax.relim()
        fig.canvas.draw()
        plt.savefig('results/spectrum_distribution_example_1.png')
        
        
        display(Markdown("<h3>Delay and controller's gains</h3>"))
        print("tau = ",num_tau)
        print("n0 = ",num_n_0)
        print("nr_0 = ",num_nr_0)
        print("d_0 = ",num_d_0)
        print("dr_0 = ",num_dr_0)

        display(Markdown("<h3>Ouput equation</h3>"))
        print("Fp = ", delta_poly)
        
        display(Markdown("<h3>Roots</h3>"))
        print(roots)
        
        intprogress_1_example_1.value = 9
        
        
        

def button_ready_to_compute_clicked_example_1(self):

    #input_interface_reset_mid_generic()

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Example 1 analysis compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    #n_mid_generic = slider_n_mid_generic.value
    #m_mid_generic = slider_m_mid_generic.value
    #s0_mid_generic = floattext_s0_mid_generic.value
    #tau_mid_generic = floattext_tau_mid_generic.value
    #n_value_mid_generic = slider_n_mid_generic.value
    #m_value_mid_generic = slider_m_mid_generic.value
    #s0_value_mid_generic = floattext_s0_mid_generic.value
    #tau_value_mid_generic = floattext_tau_mid_generic.value
    
    
    intprogress_1_example_1.value = 3
    
    computation_2_example_1()
    
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Example 1 analysis compute done !',
                buttons: {
                    'OK': {}
                }
        });})"""))

    
    intprogress_1_example_1.value = 10


button_ready_to_compute_example_1 = widgets.Button(
    description='Compute analysis', layout=widgets.Layout(width='auto'),
    disabled=True, button_style='warning',
    icon='play-circle')

button_ready_to_compute_example_1.on_click(
    button_ready_to_compute_clicked_example_1)


intprogress_1_example_1 = widgets.IntProgress(
    layout=widgets.Layout(width='auto'), value=0, min=0, max=10,
    description='Loading  ', style={'bar_color': '#ff9800'},
    orientation='horizontal')
layout_validate_1_example_1 = widgets.AppLayout(
    header=None,
    left_sidebar=button_ready_to_compute_example_1,
    center=None,
    right_sidebar=intprogress_1_example_1,
    footer=None)

# -----------------------------------------------------------------------------
# FUNCTION FOR FRONT-END
# -----------------------------------------------------------------------------
# Generic MID tab

def input_interface_reset_mid_generic():
    """ Resets the input interface of generic mid part by cleaning up
    generated files, figures, outputs, progress bars, graying out (disabled)
    buttons and checkboxes. """

    # Output
    output_compute_analysis_mid_generic.clear_output()
    output_time_simulation_computation_mid_generic.clear_output()
    output_file_link_mid_generic.clear_output()

    # Generated files and figures
    if path.exists("results/time_simulation_mid_generic.png"):
        os.remove("results/time_simulation_mid_generic.png")
    if path.exists("results/spectrum_distribution_mid_generic.png"):
        os.remove("results/spectrum_distribution_mid_generic.png")
    if os.path.exists('P3D_Online_Report_Generic_MID.pdf'):
        os.remove('P3D_Online_Report_Generic_MID.pdf')

    # Progress bar
    intprogress_1_mid_generic.value = 0
    intprogress_2_mid_generic.value = 0

    # Checkbox
    ck_pdf_report_analysis_mid_generic.value  = False
    ck_pdf_report_analysis_mid_generic.disabled = True
    ck_pdf_report_time_simulation_mid_generic.value = False
    ck_pdf_report_time_simulation_mid_generic.disabled = True

    # Button
    button_ready_to_simulation_time_mid_generic.disabled = True
    export_pdf_button_mid_generic.disabled = True


def computation_1_mid_generic(n, m, s0, value_tau):
    output_compute_analysis_mid_generic.clear_output()
    global eq_mid_generic
    global factorization_integral_latex_equation_mid_generic
    global factorization_1f1_latex_equation_mid_generic
    global a_coeff_chain_mid_generic
    global b_coeff_chain_mid_generic
    global a_coeff_mid_generic
    global b_coeff_mid_generic

    with output_compute_analysis_mid_generic:

        intprogress_1_mid_generic.value = 6
        display(Markdown("<h3>Spectrum distribution</h3>"))
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        ax.set_title("(P3delta) Spectrum distribution")
        ax.set_xlabel(r"$\Re(s)$")
        ax.set_ylabel(r"$\Im(s)$")
        ax.axhline(linewidth=2, color='black', zorder=2)
        ax.axvline(linewidth=2, color='black', zorder=2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder=2,
                   label='Unstable system')
        ax.grid()

        line, = ax.plot([], [], 'o', color='steelblue', zorder=3,
                        label='Roots')
        xr, yr, eq_mid_generic, a_coeff_mid_generic, b_coeff_mid_generic = compute_root_spectrum_mid_generic(n, m, s0,
                                                                   value_tau)
        xmax = max(xr)
        plt.autoscale()
        line.set_data(xr, yr)
        xmax_round = round(xmax, 2)
        line2, = ax.plot([], [], '*', markersize=13, color='#ff9900', zorder=3,
                         label='Rightmost root (' + str(
                             xmax_round) + ';' + '0' + ')')
        line2.set_data(xmax, 0)
        ax.legend(loc='upper right')
        ax.relim()
        fig.canvas.draw()
        plt.savefig('results/spectrum_distribution_mid_generic.png')

        display(Markdown("<h3>Parameters</h3>"))
        print("n = ", slider_n_mid_generic.value, ", m = ",
              slider_m_mid_generic.value, ", s0 = ",
              floattext_s0_mid_generic.value, ", tau = ",
              floattext_tau_mid_generic.value)
        a_coeff_chain_mid_generic = ""
        b_coeff_chain_mid_generic = ""
        i = 0
        for a_coeff in a_coeff_mid_generic:
            a_coeff_chain_mid_generic = a_coeff_chain_mid_generic + "a" + str(
                i) + " = " + str(
                a_coeff) + ", "
            i = i + 1
        print(a_coeff_chain_mid_generic)
        i = 0
        for b_coeff in b_coeff_mid_generic:
            b_coeff_chain_mid_generic = b_coeff_chain_mid_generic + "b" + str(
                i) + " = " + str(
                b_coeff) + ", "
            i = i + 1
        print(b_coeff_chain_mid_generic)

        display(Markdown("<h3>Ouput equation</h3>"))
        print("Fp = ", eq_mid_generic)
        display(Markdown("<h3>Factorized integral equation</h3>"))
        factorization_integral_latex_equation_mid_generic = \
            factorization_integral_latex(n, m, s0, value_tau)
        display(Markdown(factorization_integral_latex_equation_mid_generic))
        display(Markdown("<h3>Hypergeometric factorization</h3>"))
        factorization_1f1_latex_equation_mid_generic = factorization_1f1_latex(
            n, m, s0, value_tau)
        display(Markdown(factorization_1f1_latex_equation_mid_generic))
        intprogress_1_mid_generic.value = 9


def button_ready_to_compute_clicked_mid_generic(self):

    input_interface_reset_mid_generic()

    global n_value_mid_generic
    global m_value_mid_generic
    global s0_value_mid_generic
    global tau_value_mid_generic

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic MID analysis compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    n_mid_generic = slider_n_mid_generic.value
    m_mid_generic = slider_m_mid_generic.value
    s0_mid_generic = floattext_s0_mid_generic.value
    tau_mid_generic = floattext_tau_mid_generic.value
    n_value_mid_generic = slider_n_mid_generic.value
    m_value_mid_generic = slider_m_mid_generic.value
    s0_value_mid_generic = floattext_s0_mid_generic.value
    tau_value_mid_generic = floattext_tau_mid_generic.value
    intprogress_1_mid_generic.value = 3
    computation_1_mid_generic(n_mid_generic, m_mid_generic, s0_mid_generic,
                              tau_mid_generic)

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic MID analysis compute done ! Time simulation and \
                pdf export are available.',
                buttons: {
                    'OK': {}
                }
        });})"""))

    ck_pdf_report_analysis_mid_generic.value = True
    ck_pdf_report_analysis_mid_generic.disabled = True

    intprogress_1_mid_generic.value = 10
    constant_constant_mid_generic.disabled = False
    floattext_simulation_time_mid_generic.disabled = False
    inttext_n_iter_mid_generic.disabled = False
    dropdown_mid_generic.disabled = False
    button_ready_to_simulation_time_mid_generic.disabled = False
    export_pdf_button_mid_generic.disabled = False


button_ready_to_compute_mid_generic.on_click(
    button_ready_to_compute_clicked_mid_generic)


def computation_2_mid_generic(degn, delay, acoef, bcoef, t_final, n_iter,
                              init_type, init_args):
    output_time_simulation_computation_mid_generic.clear_output()

    with output_time_simulation_computation_mid_generic:
        display(Markdown("<h3>Time simulation plot</h3>"))

        time_initsolution, sol_initsolution = initialsolution(degn, delay,
                                                              acoef, bcoef,
                                                              t_final, n_iter,
                                                              init_type,
                                                              init_args)
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        plt.plot(time_initsolution, sol_initsolution[0, :], '#f84434')
        ax.grid()
        plt.xlabel(r"$t$ (in second)", fontsize=12)
        plt.ylabel(r"$y(t)$")
        plt.title("(P3delta) Time simulation")
        plt.savefig('results/time_simulation_mid_generic.png')
        intprogress_2_mid_generic.value = 9


def button_ready_to_simulation_time_clicked_mid_generic(self):

    intprogress_2_mid_generic.value = 0
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic MID time simulation compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    degn = slider_n_mid_generic.value
    delay = floattext_tau_mid_generic.value
    acoef = a_coeff_mid_generic
    bcoef = b_coeff_mid_generic
    t_final = floattext_simulation_time_mid_generic.value
    n_iter = inttext_n_iter_mid_generic.value
    init_type = dropdown_mid_generic.value
    intprogress_2_mid_generic.value = 3

    if init_type == 'Constant':
        init_args = [constant_constant_mid_generic.value]

    elif init_type == 'Polynomial':
        init_args = [int(polynomial_degree_mid_generic.value)]

        for i in range(int(polynomial_degree_mid_generic.value) + 1):
            init_args.append(
                items_floattext_polynomial_coef_mid_generic[i].value)

    elif init_type == 'Exponential':
        init_args = [exponential_gain_mid_generic.value,
                     exponential_exponant_mid_generic.value]

    elif init_type == 'Trigonometric':
        init_args = [trigonometric_amplitude_mid_generic.value,
                     trigonometric_frequency_mid_generic.value,
                     trigonometric_phase_mid_generic.value]

    intprogress_2_mid_generic.value = 6
    computation_2_mid_generic(degn, delay, acoef, bcoef, t_final, n_iter,
                              init_type, init_args)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic MID time simulation compute done !',
                buttons: {
                    'OK': {}
                }
        });})"""))

    ck_pdf_report_time_simulation_mid_generic.value = True
    ck_pdf_report_time_simulation_mid_generic.disabled = False
    intprogress_2_mid_generic.value = 10


button_ready_to_simulation_time_mid_generic.on_click(
    button_ready_to_simulation_time_clicked_mid_generic)


# -----------------------------------------------------------------------------
# Control oriented MID tab

def input_interface_reset_mid_co():
    """ Resets the input interface of co mid part by cleaning up
    generated files, figures, outputs, progress bars, graying out (disabled)
    buttons and checkboxes. """

    # Output
    output_admissibilite_computation_mid_co.clear_output()
    output_compute_analysis_mid_co.clear_output()
    output_sensibilite_computation_mid_co.clear_output()
    output_time_simulation_computation_mid_co.clear_output()
    output_file_link_mid_co.clear_output()

    # Generated files and figures
    if path.exists("results/admissibility_plot.png"):
        os.remove("results/admissibility_plot.png")
    if path.exists("results/spectrum_distribution_mid_co.png"):
        os.remove("results/spectrum_distribution_mid_co.png")
    if path.exists("results/time_simulation_mid_co.png"):
        os.remove("results/time_simulation_mid_co.png")
    if os.path.exists('P3D_Online_Report_Control_Oriented_MID.pdf'):
        os.remove('P3D_Online_Report_Control_Oriented_MID.pdf')

    # Progress bar
    intprogress_1_mid_co.value = 0
    intprogress_2_mid_co.value = 0
    intprogress_3_mid_co.value = 0

    # Checkbox
    ck_pdf_report_analysis_mid_co.value  = False
    ck_pdf_report_analysis_mid_co.disabled = True
    ck_pdf_report_sensitivity_mid_co.value  = False
    ck_pdf_report_sensitivity_mid_co.disabled = True
    ck_pdf_report_time_simulation_mid_co.value = False
    ck_pdf_report_time_simulation_mid_co.disabled = True

    # Button
    button_ready_to_compute_mid_co.disabled = True
    button_ready_to_sensitivity_mid_co.disabled = True
    button_ready_to_simulation_time_mid_co.disabled = True
    export_pdf_button_mid_co.disabled = True
    floattext_step_sensitivity_mid_co.disabled = True
    inttext_n_iter_sensitivity_mid_co.disabled = True

    # Other widgets
    constant_constant_mid_co.disabled = True
    floattext_simulation_time_mid_co.disabled = True
    inttext_n_iter_mid_co.disabled = True
    dropdown_mid_co.disabled = True
    dropdown_mid_co.value = 'Constant'
    

def computation_1_mid_co(n, m, a_coeff_mid_co):

    global liste_s0_sensitivity_max
    global liste_s0_sensitivity_min
    global liste_tau_sensitivity_max
    global liste_tau_sensitivity_min

    output_admissibilite_computation_mid_co.clear_output()

    with output_admissibilite_computation_mid_co:
        s0range, taurange, polyadm, s, tau, q, avalue, sysderivatif, alpha, \
            a, sol = compute_admissibilite(n, m, a_coeff_mid_co)

        if togglebuttons_limits_mid_co.value == 's0 and tau custom limits':
            s0range = np.linspace(boundeddloattext_s0_limits_min_mid_co.value,
                                  boundeddloattext_s0_limits_max_mid_co.value,
                                  num=1000)
            taurange = np.linspace(
                boundeddloattext_tau_limits_min_mid_co.value,
                boundeddloattext_tau_limits_max_mid_co.value,
                num=1000)

        elif togglebuttons_limits_mid_co.value == 's0 and tau defaut limits':
            s0range = np.linspace(-16, 0, num=200)
            taurange = np.linspace(0, 10, num=200)

        func = sp.lambdify([s, tau], polyadm)

        display(Markdown("<h3>Admissibility plot</h3>"))

        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        x, y = np.meshgrid(s0range, taurange)
        z = func(x, y)

        cs = ax.contour(x, y, z, [0], colors='#ff9c28')
        ax.grid()

        plt.xlabel(r"$s_0$")
        plt.ylabel(r"$\tau$")
        plt.title("(P3delta) Admissibility plot")
        plt.savefig('results/admissibility_plot.png')

        p = cs.collections[0].get_paths()[0]
        v = p.vertices
        liste_s0_sensitivity = v[:, 0].copy()
        liste_tau_sensitivity = v[:, 1].copy()

        liste_s0_sensitivity_max = round(np.max(liste_s0_sensitivity), 5)
        liste_s0_sensitivity_min = round(np.min(liste_s0_sensitivity), 5)
        liste_tau_sensitivity_max = round(np.max(liste_tau_sensitivity), 5)
        liste_tau_sensitivity_min = round(np.min(liste_tau_sensitivity), 5)

        display(Markdown("<h3>s0 and tau stable system limits</h3>"))
        display(Markdown(
            r"\$" + str(liste_s0_sensitivity_min) + "\\leq s_0 \\leq " + str(
                liste_s0_sensitivity_max) + "$"))
        display(Markdown(
            r"\$" + str(liste_tau_sensitivity_min) + "\\leq tau \\leq " + str(
                liste_tau_sensitivity_max) + "$"))

        if liste_tau_sensitivity_max <= floattext_s0_tau_mid_co.min:
            floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min
            floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max

        elif liste_tau_sensitivity_max > floattext_s0_tau_mid_co.min:
            floattext_s0_tau_mid_co.max = liste_tau_sensitivity_max
            floattext_s0_tau_mid_co.min = liste_tau_sensitivity_min

        floattext_s0_tau_mid_co.step = 0.01
        floattext_s0_tau_mid_co.value = 0.73
        floattext_s0_tau_mid_co.description = 'tau : '
        floattext_s0_tau_mid_co.disabled = False


def computation_2_mid_co(n, m, user_choice, user_value):
    output_compute_analysis_mid_co.clear_output()

    global a_coeff_mid_co
    global b_coeff_mid_co
    global computedtau
    global computeds0
    global eq_mid_co
    global a_coeff_chain_mid_co
    global b_coeff_chain_mid_co
    global s0_value_mid_co
    global tau_value_mid_co
    global user_choice_mid_co
    

    with output_compute_analysis_mid_co:

        display(Markdown("<h3>Spectrum distribution</h3>"))

        s0range, taurange, polyadm, s, tau, q, avalue, sysderivatif, alpha, \
            a, sol = compute_admissibilite(n, m, a_coeff_mid_co)

        finaleq = sysderivatif[-1].subs(
            {alph: alphacoef for alph, alphacoef in zip(alpha, sol)})
        finaleq = finaleq.subs({asymb: aval for asymb, aval in zip(a, avalue)})

        intprogress_1_mid_co.value = 6

        if user_choice == 'tau':

            user_choice_mid_co = user_choice
            value_tau = user_value
            sols0 = finaleq.subs({tau: value_tau})
            sols0 = sp.solve(sols0)
            computeds0 = sols0[1]
            alpha_num = sol.subs(
                {asymb: aval for asymb, aval in zip(a, avalue)})
            alpha_num = alpha_num.subs({s: computeds0})
            alpha_num = alpha_num.subs({tau: value_tau})
            alpha_num_eval = [i.evalf() for i in alpha_num]
            b_coeff_mid_co = alpha_num_eval.copy()
            
            a_coeff_mid_co = []
            for i in range(slider_n_mid_co.value):
                a_coeff_mid_co.append(items_floattext_degree_n_coef_mid_co[i].value)
            
            xr, yr, eq_mid_co, a_coeff_mid_co, b_coeff_mid_co = compute_root_spectrum_mid_co(n, m, value_tau, a_coeff_mid_co, b_coeff_mid_co)
            s0_value_mid_co = computeds0
            tau_value_mid_co = value_tau

        elif user_choice == 's0':

            user_choice_mid_co = user_choice
            value_s0 = user_value
            soltau = finaleq.subs({s: value_s0})
            soltau = sp.solve(soltau)
            computedtau = soltau[0]
            alpha_num = sol.subs(
                {asymb: aval for asymb, aval in zip(a, avalue)})
            alpha_num = alpha_num.subs({tau: computedtau})
            alpha_num = alpha_num.subs({s: value_s0})
            alpha_num_eval = [i.evalf() for i in alpha_num]
            b_coeff_mid_co = alpha_num_eval.copy()
            xr, yr, eq_mid_co, a_coeff_mid_co, b_coeff_mid_co = compute_root_spectrum_mid_co(n, m, computedtau, a_coeff_mid_co, b_coeff_mid_co)
            s0_value_mid_co = value_s0
            tau_value_mid_co = computedtau

        intprogress_1_mid_co.value = 9
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        ax.set_title("(P3delta) Spectrum distribution")
        ax.set_xlabel(r"$\Re(s)$")
        ax.set_ylabel(r"$\Im(s)$")
        ax.axhline(linewidth=2, color='black', zorder=2)
        ax.axvline(linewidth=2, color='black', zorder=2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder=2,
                   label='Unstable system')
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder=3,
                        label='Roots')
        xmax = max(xr)
        plt.autoscale()
        line.set_data(xr, yr)
        xmax_round = round(xmax, 2)
        line2, = ax.plot([], [], '*', markersize=13, color='#ff9900', zorder=3,
                         label='Rightmost root (' + str(
                             xmax_round) + ';' + '0' + ')')
        line2.set_data(xmax, 0)
        ax.legend(loc='upper right')
        ax.relim()
        fig.canvas.draw()
        plt.savefig('results/spectrum_distribution_mid_co.png')

        display(Markdown("<h3>Parameters</h3>"))

        if user_choice == 'tau':
            print("n = ", slider_n_mid_co.value, ", m = ",
                  slider_m_mid_co.value, ", s0 = ", computeds0, ", tau = ",
                  user_value)
        elif user_choice == 's0':
            print("n = ", slider_n_mid_co.value, ", m = ",
                  slider_m_mid_co.value, ", s0 = ", user_value, ", tau = ",
                  computedtau)

        a_coeff_chain_mid_co = ""
        b_coeff_chain_mid_co = ""
        i = 0
        for a_coeff in a_coeff_mid_co:
            a_coeff_chain_mid_co = a_coeff_chain_mid_co + "a" + str(
                i) + " = " + str(
                a_coeff) + ", "
            i = i + 1
        print(a_coeff_chain_mid_co)
        i = 0
        for b_coeff in b_coeff_mid_co:
            b_coeff_chain_mid_co = b_coeff_chain_mid_co + "b" + str(
                i) + " = " + str(
                b_coeff) + ", "
            i = i + 1
        print(b_coeff_chain_mid_co)

        display(Markdown("<h3>Ouput equation</h3>"))
        print("Fp = ", eq_mid_co)


def computation_3_mid_co(n, m, user_choice, user_value):
    output_sensibilite_computation_mid_co.clear_output()

    global a_coeff_mid_co
    global b_coeff_mid_co
    global computedtau

    with output_sensibilite_computation_mid_co:

        s0range, taurange, polyadm, s, tau, q, avalue, sysderivatif, alpha, \
            a, sol = compute_admissibilite(n, m, a_coeff_mid_co)
        intprogress_2_mid_co.value = 6

        if user_choice == 'tau':

            value_tau = user_value

        elif user_choice == 's0':

            if type(computedtau).__name__ == "Add":
                value_tau = computedtau.as_real_imag()[0].evalf(10)
            elif type(computedtau).__name__ == "Float":
                value_tau = float(computedtau.evalf(10))
            value_tau = round(value_tau, 5)
            
        number_iterations = inttext_n_iter_sensitivity_mid_co.value
        step = floattext_step_sensitivity_mid_co.value

        tau_sens, s0_sens, colormapper, s0_nominal = compute_sensibilite(value_tau, q, m,
                                                             s, sysderivatif,
                                                             alpha, a, tau, a_coeff_mid_co,
                                                             number_iterations, step)
        intprogress_2_mid_co.value = 9
        display(Markdown("<h3>Sensitivity plot</h3>"))
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        alpha_sens = b_coeff_mid_co.copy()
        for i in range(len(tau_sens)):
            xroot, yroot, func = getroots(1, q, tau_sens[i], avalue,
                                          alpha_sens, [-5, 5], [-5, 5], s, a,
                                          alpha, tau)
            ax.scatter(xroot, yroot, c=len(xroot) * [
                colormapper.to_rgba(float(tau_sens[i]))])
        
        s0_nominal_round = round(s0_nominal, 2)
        line2, = ax.plot([], [], '*', markersize=13, color='#80ffb5', zorder=3,
                         label='Nominal Rightmost root (' + str(
                             s0_nominal_round) + ';' + '0' + ')')
        line2.set_data(s0_nominal, 0)
        ax.legend(loc='upper right')
        
        ax.grid()
        ax.set_axisbelow(True)
        fig.colorbar(colormapper)

        ax.set_xlabel(r"$Re(s)$")
        ax.set_ylabel(r"$Im(s)$")
        ax.set_title(
            r"(P3delta) Sensitivity plot, $\tau \in $" +
            f"[{tau_sens[0]}, {tau_sens[-1]}]")
        tau_sens.sort()
        plt.savefig('results/sensitivity.png')


def computation_4_mid_co(degn, delay, acoef, bcoef, t_final, n_iter, init_type,
                         init_args):
    output_time_simulation_computation_mid_co.clear_output()

    with output_time_simulation_computation_mid_co:
        display(Markdown("<h3>Time simulation plot</h3>"))
        time_initsolution, sol_initsolution = initialsolution(degn, delay,
                                                              acoef, bcoef,
                                                              t_final, n_iter,
                                                              init_type,
                                                              init_args)
        intprogress_3_mid_co.value = 6
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        plt.plot(time_initsolution, sol_initsolution[0, :], '#f84434')
        ax.grid()
        plt.xlabel(r"$t$ (in second)", fontsize=12)
        plt.ylabel(r"$y(t)$")
        plt.title("(P3delta) Time simulation")
        plt.savefig('results/time_simulation_mid_co.png')
        intprogress_3_mid_co.value = 9


def button_confirm_system_coeff_clicked_mid_co(self):

    input_interface_reset_mid_co()

    global a_coeff_mid_co
    global n_value_mid_co
    global m_value_mid_co

    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    n_value_mid_co = slider_n_mid_co.value
    m_value_mid_co = slider_m_mid_co.value
    a_coeff_mid_co = []

    for i in range(slider_n_mid_co.value):
        a_coeff_mid_co.append(items_floattext_degree_n_coef_mid_co[i].value)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Coefficients of control oriented MID analysis confirmed !',
                buttons: {
                    'OK': {}
                }
        });})"""))
    computation_1_mid_co(n_mid_co, m_mid_co, a_coeff_mid_co)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Admissibility compute done ! Control oriented MID analysis \
                is available.',
                buttons: {
                    'OK': {}
                }
        });})"""))
    togglebuttons_mid_co.disabled = False
    togglebuttons_stable_limits_mid_co.disabled = False
    button_ready_to_compute_mid_co.disabled = False


def button_ready_to_compute_clicked_mid_co(self):
    global a_coeff_mid_co

    intprogress_1_mid_co.value = 0
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID analysis compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    user_choice = togglebuttons_mid_co.value
    user_value = floattext_s0_tau_mid_co.value
    intprogress_1_mid_co.value = 3

    computation_2_mid_co(n_mid_co, m_mid_co, user_choice, user_value)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID analysis compute done ! Tau \
                sensitivity, and time simulation are available.',
                buttons: {
                    'OK': {}
                }
        });})"""))

    ck_pdf_report_analysis_mid_co.value = True
    ck_pdf_report_analysis_mid_co.disabled = True

    intprogress_1_mid_co.value = 10
    export_pdf_button_mid_co.disabled = False
    button_ready_to_sensitivity_mid_co.disabled = False
    constant_constant_mid_co.disabled = False
    floattext_simulation_time_mid_co.disabled = False
    inttext_n_iter_mid_co.disabled = False
    dropdown_mid_co.disabled = False
    button_ready_to_simulation_time_mid_co.disabled = False
    floattext_step_sensitivity_mid_co.disabled = False
    inttext_n_iter_sensitivity_mid_co.disabled = False


def button_ready_to_sensitivity_clicked_mid_co(self):
    intprogress_2_mid_co.value = 0
    n_mid_co = slider_n_mid_co.value
    m_mid_co = slider_m_mid_co.value
    user_choice = togglebuttons_mid_co.value
    user_value = floattext_s0_tau_mid_co.value

    intprogress_2_mid_co.value = 3
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID tau sensitivity compute in \
                progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    computation_3_mid_co(n_mid_co, m_mid_co, user_choice, user_value)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID tau sensitivity compute done !',
                buttons: {
                    'OK': {}
                }
        });})"""))

    intprogress_2_mid_co.value = 10
    ck_pdf_report_sensitivity_mid_co.value = True
    ck_pdf_report_sensitivity_mid_co.disabled = False


def button_ready_to_simulation_time_clicked_mid_co(self):
    intprogress_3_mid_co.value = 0
    display(Javascript("""require(["base/js/dialog"],function(dialog) {
        dialog.modal({
            title: 'Information',
            body: 'Control oriented MID time simulation compute \
            in progress...',
            buttons: {
                'OK': {}
            }
    });})"""))
    degn = slider_n_mid_co.value

    if togglebuttons_mid_co.value == 's0':
        delay = computedtau

    elif togglebuttons_mid_co.value == 'tau':
        delay = floattext_s0_tau_mid_co.value

    acoef = a_coeff_mid_co
    bcoef = b_coeff_mid_co
    t_final = floattext_simulation_time_mid_co.value
    n_iter = inttext_n_iter_mid_co.value
    init_type = dropdown_mid_co.value

    if init_type == 'Constant':
        init_args = [constant_constant_mid_co.value]

    elif init_type == 'Polynomial':
        init_args = [int(polynomial_degree_mid_co.value)]

        for i in range(int(polynomial_degree_mid_co.value) + 1):
            init_args.append(items_floattext_polynomial_coef_mid_co[i].value)

    elif init_type == 'Exponential':
        init_args = [exponential_gain_mid_co.value,
                     exponential_exponant_mid_co.value]

    elif init_type == 'Trigonometric':
        init_args = [trigonometric_amplitude_mid_co.value,
                     trigonometric_frequency_mid_co.value,
                     trigonometric_phase_mid_co.value]

    intprogress_3_mid_co.value = 3
    computation_4_mid_co(degn, delay, acoef, bcoef, t_final, n_iter, init_type,
                         init_args)

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID time simulation compute done !',
                buttons: {
                    'OK': {}
                }
        });})"""))

    intprogress_3_mid_co.value = 10
    ck_pdf_report_time_simulation_mid_co.value = True
    ck_pdf_report_time_simulation_mid_co.disabled = False


button_ready_to_simulation_time_mid_co.on_click(
    button_ready_to_simulation_time_clicked_mid_co)

button_ready_to_compute_mid_co.on_click(button_ready_to_compute_clicked_mid_co)

button_ready_to_sensitivity_mid_co.on_click(
    button_ready_to_sensitivity_clicked_mid_co)

button_confirm_system_coeff_mid_co.on_click(
    button_confirm_system_coeff_clicked_mid_co)


# -----------------------------------------------------------------------------
# Generic CRRID tab


def input_interface_reset_crrid_generic():
    """ Resets the input interface of generic crrid part by cleaning up
    generated files, figures, outputs, progress bars, graying out (disabled)
    buttons and checkboxes. """

    # Output
    output_compute_analysis_crrid_generic.clear_output()
    output_time_simulation_computation_crrid_generic.clear_output()
    output_file_link_crrid_generic.clear_output()

    # Generated files and figures
    if path.exists("results/time_simulation_crrid_generic.png"):
        os.remove("results/time_simulation_crrid_generic.png")
    if path.exists("results/spectrum_distribution_crrid_generic.png"):
        os.remove("results/spectrum_distribution_crrid_generic.png")
    if os.path.exists('P3D_Online_Report_Generic_CRRID.pdf'):
        os.remove('P3D_Online_Report_Generic_CRRID.pdf')

    # Progress bar
    intprogress_1_crrid_generic.value = 0
    intprogress_2_crrid_generic.value = 0

    # Checkbox
    ck_pdf_report_analysis_crrid_generic.value  = False
    ck_pdf_report_analysis_crrid_generic.disabled = True
    ck_pdf_report_time_simulation_crrid_generic.value = False
    ck_pdf_report_time_simulation_crrid_generic.disabled = True

    # Button
    button_ready_to_simulation_time_crrid_generic.disabled = True
    export_pdf_button_crrid_generic.disabled = True

def computation_1_crrid_generic(n, m, s0, d, value_tau):

    output_compute_analysis_crrid_generic.clear_output()
    global eq_crrid_generic
    global a_coeff_chain_crrid_generic
    global b_coeff_chain_crrid_generic
    global a_coeff_crrid_generic
    global b_coeff_crrid_generic

    with output_compute_analysis_crrid_generic:

        display(Markdown("<h3>Spectrum distribution</h3>"))
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        ax.set_title(
            "(P3delta) Spectrum distribution with equidistributed rightmost "
            "spectral values")
        ax.set_xlabel(r"Re$(s)$")
        ax.set_ylabel(r"Im$(s)$")
        ax.axhline(linewidth=2, color='black', zorder=2)
        ax.axvline(linewidth=2, color='black', zorder=2)
        ax.axvspan(0, 1, alpha=0.5, color='red', zorder=2,
                   label='Unstable system')
        ax.grid()
        line, = ax.plot([], [], 'o', color='steelblue', zorder=3,
                        label='Roots')
        xr, yr, eq_crrid_generic, a_coeff_crrid_generic, b_coeff_crrid_generic = compute_crrid_generic_root(n, m, s0,
                                                              d, value_tau, a_coeff_crrid_generic, b_coeff_crrid_generic)
        plt.autoscale()
        line.set_data(xr, yr)
        xmax2 = max(xr)
        xmax2_round = round(xmax2, 2)
        line2, = ax.plot([], [], '*', markersize=13, color='#ff9900', zorder=3,
                         label='Rightmost root (' + str(
                             xmax2_round) + ';' + '0' + ')')
        line2.set_data(xmax2, 0)
        ax.legend(loc='upper right')
        ax.relim()
        fig.canvas.draw()
        plt.savefig('results/spectrum_distribution_crrid_generic.png')
        intprogress_1_crrid_generic.value = 6

        display(Markdown("<h3>Parameters</h3>"))
        print("n = ", slider_n_crrid_generic.value, ", m = ",
              slider_m_crrid_generic.value, ", s0 = ",
              floattext_s0_crrid_generic.value, ", tau = ",
              floattext_tau_crrid_generic.value)
        a_coeff_chain_crrid_generic = ""
        b_coeff_chain_crrid_generic = ""
        i = 0
        for a_coeff in a_coeff_crrid_generic:
            a_coeff_chain_crrid_generic = a_coeff_chain_crrid_generic + \
                                          "a" + str(i) + " = " + str(a_coeff) \
                                          + ", "
            i = i + 1
        print(a_coeff_chain_crrid_generic)
        i = 0
        for b_coeff in b_coeff_crrid_generic:
            b_coeff_chain_crrid_generic = b_coeff_chain_crrid_generic + \
                                          "b" + str(i) + " = " + str(b_coeff) \
                                          + ", "
            i = i + 1
        print(b_coeff_chain_crrid_generic)
        intprogress_1_crrid_generic.value = 9

        display(Markdown("<h3>Ouput equation</h3>"))
        print("Fp = ", eq_crrid_generic)


def button_ready_to_compute_clicked_crrid_generic(self):

    input_interface_reset_crrid_generic()

    global n_value_crrid_generic
    global m_value_crrid_generic
    global s0_value_crrid_generic
    global d_value_crrid_generic
    global tau_value_crrid_generic

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic CRRID analysis compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    n_crrid_generic = slider_n_crrid_generic.value
    m_crrid_generic = slider_m_crrid_generic.value
    s0_crrid_generic = floattext_s0_crrid_generic.value
    d_crrid_generic = floattext_d_crrid_generic.value
    tau_crrid_generic = floattext_tau_crrid_generic.value
    n_value_crrid_generic = slider_n_crrid_generic.value
    m_value_crrid_generic = slider_m_crrid_generic.value
    s0_value_crrid_generic = floattext_s0_crrid_generic.value
    d_value_crrid_generic = floattext_d_crrid_generic.value
    tau_value_crrid_generic = floattext_tau_crrid_generic.value

    intprogress_1_crrid_generic.value = 3
    computation_1_crrid_generic(n_crrid_generic, m_crrid_generic,
                                s0_crrid_generic, d_crrid_generic,
                                tau_crrid_generic)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic CRRID analysis compute done ! Time simulation \
                and pdf export are available.',
                buttons: {
                    'OK': {}
                }
        });})"""))

    intprogress_1_crrid_generic.value = 10
    ck_pdf_report_analysis_crrid_generic.value = True
    ck_pdf_report_analysis_crrid_generic.disabled = True

    constant_constant_crrid_generic.disabled = False
    floattext_simulation_time_crrid_generic.disabled = False
    inttext_n_iter_crrid_generic.disabled = False
    dropdown_crrid_generic.disabled = False
    button_ready_to_simulation_time_crrid_generic.disabled = False
    export_pdf_button_crrid_generic.disabled = False


button_ready_to_compute_crrid_generic.on_click(
    button_ready_to_compute_clicked_crrid_generic)


def computation_2_crrid_generic(degn, delay, acoef, bcoef, t_final, n_iter,
                                init_type, init_args):
    output_time_simulation_computation_crrid_generic.clear_output()

    with output_time_simulation_computation_crrid_generic:
        display(Markdown("<h3>Time simulation plot</h3>"))
        intprogress_2_crrid_generic.value = 6
        time_initsolution, sol_initsolution = initialsolution(degn, delay,
                                                              acoef, bcoef,
                                                              t_final, n_iter,
                                                              init_type,
                                                              init_args)
        fig, ax = plt.subplots(figsize=(9, 5))
        watermark2(ax, fig)
        plt.plot(time_initsolution, sol_initsolution[0, :], '#f84434')
        ax.grid()
        plt.xlabel(r"$t$ (in second)", fontsize=12)
        plt.ylabel(r"$y(t)$")
        plt.title("(P3delta) Time simulation")
        intprogress_2_crrid_generic.value = 9
        plt.savefig('results/time_simulation_crrid_generic.png')


def button_ready_to_simulation_time_clicked_crrid_generic(self):
    intprogress_2_crrid_generic.value = 0
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic CRRID time simulation compute in progress...',
                buttons: {
                    'OK': {}
                }
        });})"""))
    degn = slider_n_crrid_generic.value
    delay = floattext_tau_crrid_generic.value
    acoef = a_coeff_crrid_generic
    bcoef = b_coeff_crrid_generic
    t_final = floattext_simulation_time_crrid_generic.value
    n_iter = inttext_n_iter_crrid_generic.value
    init_type = dropdown_crrid_generic.value

    if init_type == 'Constant':
        init_args = [constant_constant_crrid_generic.value]

    elif init_type == 'Polynomial':
        init_args = [int(polynomial_degree_crrid_generic.value)]

        for i in range(int(polynomial_degree_crrid_generic.value) + 1):
            init_args.append(
                items_floattext_polynomial_coef_crrid_generic[i].value)

    elif init_type == 'Exponential':
        init_args = [exponential_gain_crrid_generic.value,
                     exponential_exponant_crrid_generic.value]

    elif init_type == 'Trigonometric':
        init_args = [trigonometric_amplitude_crrid_generic.value,
                     trigonometric_frequency_crrid_generic.value,
                     trigonometric_phase_crrid_generic.value]
    intprogress_2_crrid_generic.value = 3
    computation_2_crrid_generic(degn, delay, acoef, bcoef, t_final, n_iter,
                                init_type, init_args)
    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic CRRID time simulation compute done ! ',
                buttons: {
                    'OK': {}
                }
        });})"""))

    intprogress_2_crrid_generic.value = 10
    ck_pdf_report_time_simulation_crrid_generic.value = True
    ck_pdf_report_time_simulation_crrid_generic.disabled = False


button_ready_to_simulation_time_crrid_generic.on_click(
    button_ready_to_simulation_time_clicked_crrid_generic)

# -----------------------------------------------------------------------------
# PDF report tab

def generation_checkbox(name='checkbox', bool_1=True, bool_2=True):
    """ Create a checkbox widget.
    """
    checkbox = widgets.Checkbox(
        description=name,
        value=bool_1,
        disabled=bool_2)

    return checkbox

ck_pdf_report_analysis_mid_generic = generation_checkbox(
    "Analysis", False, True)
ck_pdf_report_time_simulation_mid_generic = generation_checkbox(
    "Time simulation", False, True)

ck_pdf_report_analysis_mid_co = generation_checkbox(
    "Analysis", False, True)
ck_pdf_report_sensitivity_mid_co = generation_checkbox(
    "Sensitivity", False, True)
ck_pdf_report_time_simulation_mid_co = generation_checkbox(
    "Time simulation", False, True)

ck_pdf_report_analysis_crrid_generic = generation_checkbox(
    "Analysis", False, True)
ck_pdf_report_time_simulation_crrid_generic = generation_checkbox(
    "Time simulation", False, True)


export_pdf_button_mid_generic = widgets.Button(
    description='Export Generic MID results to PDF',
    layout=widgets.Layout(width='auto'),
    disabled=True, button_style='success',
    icon='file')


def export_pdf_button_mid_generic_clicked(self):
    date_time = datetime.utcnow()
    
    class PDF(FPDF):
        def header(self):
            self.image('images/logoP3D.png', 9, 5, 20)
            self.set_font('Helvetica', 'B', 20)
            self.cell(200, 50, 'P3delta Online', 0, 0, 'C')
            self.cell(18)
            self.set_font('Helvetica', '', 11)
            self.cell(0, 10,
                      'Date (UTC) : '+ str(date_time.year) +'/'+ str(date_time.month) +'/'+ str(date_time.day) + ', '+ str(date_time.hour) +':'+ str(date_time.minute) +':'+ str(date_time.second) , 0, 0, 'R')
            self.ln(40)

        def footer(self):
            self.set_y(-15)
            self.set_font('Helvetica', 'I', 8)
            self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0,
                      'C')

        def subtitle(self):
            self.set_font('Helvetica', 'B', 17)
            self.set_text_color(255, 255, 255)
            self.set_fill_color(142, 33, 67)
            self.cell(0, 8, 'Generic MID', 0, 1, 'L', 1)
            self.ln(10)

        def subsubtitle(self):
            self.set_font('Helvetica', 'B', 14)
            self.set_fill_color(255, 255, 255)
            self.set_text_color(0, 0, 0)
            self.cell(0, 6, 'Analysis', 0, 1, 'L')
            self.ln(10)

        def subsubsubtitle_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Spectrum distribution', 0, 1, 'L')
            self.ln(120)
            self.image('results/spectrum_distribution_mid_generic.png', 25, 95,
                       170, 100)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, "Order of the open-loop plant n = " + str(
                n_value_mid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, "Order of the delayed controller m = " + str(
                m_value_mid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6,
                      "Assigned decay rate s0 =  " + str(s0_value_mid_generic),
                      0, 1, 'L')
            self.ln(5)
            self.cell(20, 6,
                      "Delay value tau =  " + str(tau_value_mid_generic), 0, 1,
                      'L')
            self.ln(5)
            self.cell(20, 6, str(a_coeff_chain_mid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, str(b_coeff_chain_mid_generic), 0, 1, 'L')
            self.add_page()

        def subsubtitle_2_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Equation Fp', 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 9)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, "Fp = " + str(eq_mid_generic), 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Factorized integral equation', 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 11)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6,
                      str(factorization_integral_latex_equation_mid_generic),
                      0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Hypergeometric factorization', 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 11)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, str(factorization_1f1_latex_equation_mid_generic),
                      0, 1, 'L')
            self.ln(15)

        def subsubtitle_3_and_body(self):
            self.set_font('Helvetica', 'B', 14)
            self.cell(20, 6, 'Time simulation', 0, 1, 'L')
            self.ln(10)
            self.image('results/time_simulation_mid_generic.png', 25, 160, 170,
                       100)

    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()
    pdf.subtitle()
    if ck_pdf_report_analysis_mid_generic.value :
        pdf.subsubtitle()
        pdf.subsubsubtitle_and_body()
        pdf.subsubtitle_2_and_body()
    if ck_pdf_report_time_simulation_mid_generic.value :
        pdf.subsubtitle_3_and_body()

    pdf.output('P3D_Online_Report_Generic_MID.pdf', 'F')

    # Link for the PDF file
    output_file_link_mid_generic.clear_output()
    with output_file_link_mid_generic:
        display(FileLink('P3D_Online_Report_Generic_MID.pdf'))

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Generic MID results export done ! ',
                buttons: {
                    'OK': {}
                }
        });})"""))


export_pdf_button_mid_generic.on_click(
    export_pdf_button_mid_generic_clicked)

export_pdf_button_crrid_generic = widgets.Button(
    description='Export Generic CRRID results to PDF',
    layout=widgets.Layout(width='auto'),
    disabled=True, button_style='success',
    icon='file')


def export_pdf_button_crrid_generic_clicked(self):
    date_time = datetime.utcnow()
    
    class PDF(FPDF):
        def header(self):
            self.image('images/logoP3D.png', 9, 5, 20)
            self.set_font('Helvetica', 'B', 20)
            self.cell(200, 50, 'P3delta Online', 0, 0, 'C')
            self.cell(18)
            self.set_font('Helvetica', '', 11)
            self.cell(0, 10,
                      'Date (UTC) : '+ str(date_time.year) +'/'+ str(date_time.month) +'/'+ str(date_time.day) + ', '+ str(date_time.hour) +':'+ str(date_time.minute) +':'+ str(date_time.second) , 0, 0, 'R')
            self.ln(40)

        def footer(self):
            self.set_y(-15)
            self.set_font('Helvetica', 'I', 8)
            self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0,
                      'C')

        def subtitle(self):
            self.set_font('Helvetica', 'B', 17)
            self.set_text_color(255, 255, 255)
            self.set_fill_color(142, 33, 67)
            self.cell(0, 8, 'Generic CRRID', 0, 1, 'L', 1)
            self.ln(10)

        def subsubtitle(self):
            self.set_font('Helvetica', 'B', 14)
            self.set_fill_color(255, 255, 255)
            self.set_text_color(0, 0, 0)
            self.cell(0, 6, 'Analysis', 0, 1, 'L')
            self.ln(10)

        def subsubsubtitle_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Spectrum distribution', 0, 1, 'L')
            self.ln(100)
            self.image('results/spectrum_distribution_crrid_generic.png', 25,
                       90, 170, 100)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, "Order of the open-loop plant n = " + str(
                n_value_crrid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, "Order of the delayed controller m = " + str(
                m_value_crrid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, "Assigned decay rate s0 =  " + str(
                s0_value_crrid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, "Step d between each s0 value =  " + str(
                d_value_crrid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6,
                      "Delay value tau =  " + str(tau_value_crrid_generic), 0,
                      1, 'L')
            self.ln(5)
            self.cell(20, 6, str(a_coeff_chain_crrid_generic), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, str(b_coeff_chain_crrid_generic), 0, 1, 'L')

        def subsubtitle_2_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.ln(100)
            self.cell(20, 6, 'Equation Fp', 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 9)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, "Fp = " + str(eq_crrid_generic), 0, 1, 'L')
            self.ln(15)

        def subsubtitle_3_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Time simulation', 0, 1, 'L')
            self.ln(10)
            self.image('results/time_simulation_crrid_generic.png', 25, 100,
                       170, 100)

    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()
    pdf.subtitle()
    if ck_pdf_report_analysis_crrid_generic.value :
        pdf.subsubtitle()
        pdf.subsubsubtitle_and_body()
        pdf.subsubtitle_2_and_body()
    if ck_pdf_report_time_simulation_crrid_generic.value :
        pdf.subsubtitle_3_and_body()

    pdf.output('P3D_Online_Report_Generic_CRRID.pdf', 'F')

    # Link for the PDF file
    output_file_link_crrid_generic.clear_output()
    with output_file_link_crrid_generic:
        display(FileLink('P3D_Online_Report_Generic_CRRID.pdf'))

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'CRRID Generic results export done !',
                buttons: {
                    'OK': {}
                }
        });})"""))


export_pdf_button_crrid_generic.on_click(
    export_pdf_button_crrid_generic_clicked)

export_pdf_button_mid_co = widgets.Button(
    description='Export Control Oriented MID results to PDF',
    layout=widgets.Layout(width='auto'),
    disabled=True, button_style='success',
    icon='file')


def export_pdf_button_mid_co_clicked(self):
    date_time = datetime.utcnow()
    
    class PDF(FPDF):
        def header(self):
            self.image('images/logoP3D.png', 9, 5, 20)
            self.set_font('Helvetica', 'B', 20)
            self.cell(200, 50, 'P3delta Online', 0, 0, 'C')
            self.cell(18)
            self.set_font('Helvetica', '', 11)
            self.cell(0, 10,
                      'Date (UTC) : '+ str(date_time.year) +'/'+ str(date_time.month) +'/'+ str(date_time.day) + ', '+ str(date_time.hour) +':'+ str(date_time.minute) +':'+ str(date_time.second) , 0, 0, 'R')
            self.ln(40)

        def footer(self):
            self.set_y(-15)
            self.set_font('Helvetica', 'I', 8)
            self.cell(0, 10, 'Page ' + str(self.page_no()) + '/{nb}', 0, 0,
                      'C')

        def subtitle(self):
            self.set_font('Helvetica', 'B', 17)
            self.set_text_color(255, 255, 255)
            self.set_fill_color(142, 33, 67)
            self.cell(0, 8, 'Control Oriented MID', 0, 1, 'L', 1)
            self.ln(10)

        def subsubtitle(self):
            self.set_font('Helvetica', 'B', 14)
            self.set_fill_color(255, 255, 255)
            self.set_text_color(0, 0, 0)
            self.cell(0, 6, 'Analysis', 0, 1, 'L')
            self.ln(10)

        def subsubtitle_0_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Admissibility plot', 0, 1, 'L')
            self.ln(120)
            self.image('results/admissibility_plot.png', 25, 95, 170, 100)
            self.set_text_color(0, 0, 0)
            self.add_page()

        def subsubsubtitle_1_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Spectrum distribution', 0, 1, 'L')
            self.ln(120)
            self.image('results/spectrum_distribution_mid_co.png', 25, 60, 170,
                       100)
            self.set_text_color(0, 0, 0)
            self.cell(20, 6, "Order of the open-loop plant n = " + str(
                n_value_mid_co), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, "Order of the delayed controller m = " + str(
                m_value_mid_co), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6,
                      "Assigned decay rate s0 =  " + str(s0_value_mid_co), 0,
                      1, 'L')
            self.ln(5)
            self.cell(20, 6, "Delay value tau =  " + str(tau_value_mid_co), 0,
                      1, 'L')
            self.ln(5)
            self.cell(20, 6, str(a_coeff_chain_mid_co), 0, 1, 'L')
            self.ln(5)
            self.cell(20, 6, str(b_coeff_chain_mid_co), 0, 1, 'L')
            self.add_page()

        def subsubtitle_2_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Equation Fp', 0, 1, 'L')
            self.ln(10)
            self.set_font('Helvetica', '', 9)
            self.set_text_color(0, 0, 0)
            self.cell(25, 6, "Fp = " + str(eq_mid_co), 0, 1, 'L')
            self.ln(15)

        def subsubtitle_3_and_body(self):
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Sensitivity', 0, 1, 'L')
            self.ln(10)
            self.image('results/sensitivity.png', 25, 100, 170, 100)
            self.ln(15)

        def subsubtitle_4_and_body(self):
            self.set_text_color(0, 0, 0)
            self.add_page()
            self.set_font('Helvetica', '', 11)
            self.set_text_color(142, 33, 67)
            self.cell(20, 6, 'Time simulation', 0, 1, 'L')
            self.ln(10)
            self.image('results/time_simulation_mid_co.png', 25, 60, 170, 100)

    pdf = PDF()
    pdf.alias_nb_pages()
    pdf.add_page()
    pdf.subtitle()
    if ck_pdf_report_analysis_mid_co.value :
        pdf.subsubtitle()
        pdf.subsubtitle_0_and_body()
        pdf.subsubsubtitle_1_and_body()
        pdf.subsubtitle_2_and_body()
    if ck_pdf_report_sensitivity_mid_co.value :
        pdf.subsubtitle_3_and_body()
    if ck_pdf_report_time_simulation_mid_co.value :
        pdf.subsubtitle_4_and_body()

    pdf.output('P3D_Online_Report_Control_Oriented_MID.pdf', 'F')

    # Link for the PDF file
    output_file_link_mid_co.clear_output()
    with output_file_link_mid_co:
        display(FileLink('P3D_Online_Report_Control_Oriented_MID.pdf'))

    if slider_notification.value == 'On' :
        display(Javascript("""require(["base/js/dialog"],function(dialog) {
            dialog.modal({
                title: 'Information',
                body: 'Control oriented MID results export done !',
                buttons: {
                    'OK': {}
                }
        });})"""))


export_pdf_button_mid_co.on_click(
    export_pdf_button_mid_co_clicked)

# -----------------------------------------------------------------------------
# USER INTERFACE OUTPUT
# -----------------------------------------------------------------------------
# Main HTML title

title_p3delta_online = widgets.HTML(
    "<h2 class = 'text-center'><font color='#8E2042'>"
    "P3delta Online</font><size='10'>")
title_settings = widgets.HTML(
    "<h2 class = 'text-center'><font color='#8E2042'>"
    "Settings</font><size='10'>")
title_analysis = widgets.HTML(
    "<h2 class = 'text-center'><font color='#ff9c28'>"
    "Analysis</font><size='10'>")
title_time_simulation = widgets.HTML(
    "<h2 class = 'text-center'><font color='#f84434'>"
    "Time simulation</font><size='10'>")
title_sensitivity = widgets.HTML(
    "<h2 class = 'text-center'><font color='#10acd8'>"
    "Sensitivity</font><size='10'>")
title_pdf_report = widgets.HTML(
    "<h2 class = 'text-center'><font color='#50ac54'>"
    "PDF Report</font><size='10'>")
title_example_1 = widgets.HTML(
    "<h2 class = 'text-center'><font color='#000000'>"
    "Active vibration control of a piezo-actuated beam</font><size='10'>")

# -----------------------------------------------------------------------------
# Label subsubtitle

separator_label = widgets.Label(
    '_________________________________________________________________________'
    '___________________________________________________________')
label_empty_space = widgets.Label(value=" ",
                                  layout=widgets.Layout(width='auto',
                                                        height='10px'))

description_label_home_1 = widgets.Label(
    'Welcome to P3delta Online ! A software for delayed \
    control design. For more information, \
    consult : ')
description_label_home_2 = widgets.HTML(
    value="<a href=https://cutt.ly/p3delta target='_blank'>P3delta website</a>")
description_label_home_3 = widgets.HTML(
    value="<a href=https://www.linkedin.com/company/p3delta target='_blank'>P3delta Linkedin page</a>")

description_label_mid_generic = widgets.Label(
    'Insert degree of polynomial n, degree of delay polynomial m, rightmost '
    'root s0 and delay tau, then push the button to compute : ')

description_label_mid_co = widgets.Label(
    'Insert degree of polynomial n, degree of delay polynomial m, '
    ' values of system coefficients : ')
description_label_2_mid_co = widgets.Label(
    'Choose default or custom limits for s0 and tau'
    ' then push the button to confirm and draw the admissibility plot : ')
description_label_3_mid_co = widgets.Label(
    'Select limits and between rightmost root s0 or delay tau using the '
    'admissibility plot, then push the button to compute : ')

description_label_1_sensitivity = widgets.Label(
    'Insert value of step and number of iteration, '
    'then push the button to compute : ')

description_label_crrid_generic = widgets.VBox([widgets.Label(
    'Insert degree of polynomial n, degree of delay polynomial m, rightmost '
    'root s0, step d between each value of s0 and delay tau, then push the '
    'button to compute :')],
    layout=widgets.Layout(
        width='100%',
        display='flex',
        align_items='center'))

time_simulation_label = widgets.Label(
    'Choose initial condition, insert time simulation t and number of iteration, '
    'then push the button to compute :')

description_pdf_report_1 = widgets.Label(
    'To generate the PDF report, '
    'select the items to be included in the PDF report and '
    'press the appropriate button :')

description_label_example_1 = widgets.Label(
    'Description du système, problème, figure, etc.')
description_label_2_example_1 = widgets.Label(
    'Insert the number of modes n of the reduced model, '
    'values of system coefficients : ')
description_label_3_example_1 = widgets.Label(
    'Choose custom limits for s0 and tau then push the button to confirm '
    'and draw the admissibility plot : ')
description_label_4_example_1 = widgets.Label(
    'Select rightmost root s0 value using the admissibility plot, '
    'then push the button to compute :')

# -----------------------------------------------------------------------------
# Subbox

box_input_mid_generic = widgets.HBox(
    [slider_n_mid_generic, slider_m_mid_generic, floattext_s0_mid_generic,
     floattext_tau_mid_generic])

box_input_mid_co = widgets.HBox(
    [degree_n_mid_co, slider_m_mid_co])
box_input_2_mid_co = widgets.HBox(
    [vbox_limits_mid_co],
    layout=widgets.Layout(width='auto'))
box_input_3_mid_co = widgets.HBox(
    [button_confirm_system_coeff_mid_co],
    layout=widgets.Layout(width='auto'))
box_input_4_mid_co = widgets.HBox(
    [layout_validate_1_mid_co], layout=widgets.Layout(width='auto'))

box_input_1_crrid_generic = AppLayout(left_sidebar=slider_n_crrid_generic,
                                      right_sidebar=slider_m_crrid_generic)
box_input_2_crrid_generic = widgets.HBox(
    [floattext_s0_crrid_generic, floattext_tau_crrid_generic,
     floattext_d_crrid_generic])

box_input_example_1 = widgets.HBox(
    [degree_n_example_1])
box_input_2_example_1 = widgets.HBox(
    [button_confirm_system_coeff_example_1],
    layout=widgets.Layout(width='auto'))
box_input_3_example_1 = widgets.HBox(
    [floattext_s0_example_1])



# -----------------------------------------------------------------------------
# Accordion

accordion_1_mid_generic = widgets.Accordion(
    children=[output_compute_analysis_mid_generic],
    titles=('Output analysis', ''))
accordion_2_mid_generic = widgets.Accordion(
    children=[output_time_simulation_computation_mid_generic],
    titles=('Output time simulation', ''))
accordion_1_mid_generic.selected_index = 0
accordion_2_mid_generic.selected_index = 0

accordion_1_mid_co = widgets.Accordion(
    children=[output_admissibilite_computation_mid_co],
    titles=('Output admissibility', ''))
accordion_2_mid_co = widgets.Accordion(
    children=[output_compute_analysis_mid_co],
    titles=('Output analysis', ''))
accordion_3_mid_co = widgets.Accordion(
    children=[output_sensibilite_computation_mid_co],
    titles=('Output sensitivity', ''))
accordion_4_mid_co = widgets.Accordion(
    children=[output_time_simulation_computation_mid_co],
    titles=('Output time simulation', ''))
accordion_1_mid_co.selected_index = 0
accordion_2_mid_co.selected_index = 0
accordion_3_mid_co.selected_index = 0
accordion_4_mid_co.selected_index = 0

accordion_1_crrid_generic = widgets.Accordion(
    children=[output_compute_analysis_crrid_generic],
    titles=('Output analysis', ''))
accordion_2_crrid_generic = widgets.Accordion(
    children=[output_time_simulation_computation_crrid_generic],
    titles=('Output time simulation', ''))
accordion_1_crrid_generic.selected_index = 0
accordion_2_crrid_generic.selected_index = 0

accordion_1_example_1 = widgets.Accordion(
    children=[output_compute_analysis_example_1],
    titles=('Output analysis', ''))
accordion_2_example_1 = widgets.Accordion(
    children=[output_admissibilite_computation_example_1],
    titles=('Output admissibility', ''))
accordion_1_example_1.selected_index = 0
accordion_2_example_1.selected_index = 0

# -----------------------------------------------------------------------------
# Image

file = open("images/description.png", "rb")
image = file.read()
description_image = widgets.Image(value=image, format='png', width=954, height=1771)

# -----------------------------------------------------------------------------
# Main structure

box_home = widgets.VBox(
    [title_p3delta_online,
     description_label_home_1,
     description_label_home_2,
     description_label_home_3,
     title_settings,
     label_empty_space,
     slider_notification,
     slider_admin_user_mode])

box_mid_generic = widgets.VBox(
    [title_analysis,
     description_label_mid_generic,
     label_empty_space,
     box_input_mid_generic,
     label_empty_space,
     layout_validate_1_mid_generic,
     label_empty_space,
     accordion_1_mid_generic,
     separator_label,
     title_time_simulation,
     time_simulation_label,
     label_empty_space,
     stacked_simulation_time_mid_generic,
     floattext_simulation_time_mid_generic,
     inttext_n_iter_mid_generic,
     label_empty_space,
     layout_validate_2_mid_generic,
     label_empty_space,
     accordion_2_mid_generic])

box_mid_co = widgets.VBox(
    [title_analysis,
     description_label_mid_co,
     label_empty_space,
     box_input_mid_co,
     label_empty_space,
     description_label_2_mid_co,
     label_empty_space,
     box_input_2_mid_co,
     label_empty_space,
     box_input_3_mid_co,
     label_empty_space,
     accordion_1_mid_co,
     label_empty_space,
     description_label_3_mid_co,
     label_empty_space,
     box_input_4_mid_co,
     label_empty_space,
     layout_validate_2_mid_co,
     label_empty_space,
     accordion_2_mid_co,
     separator_label,
     title_sensitivity,
     label_empty_space,
     description_label_1_sensitivity,
     label_empty_space,
     layout_validate_5_mid_co,
     label_empty_space,
     layout_validate_4_mid_co,
     label_empty_space,
     accordion_3_mid_co,
     separator_label,
     title_time_simulation,
     time_simulation_label,
     label_empty_space,
     stacked_simulation_time_mid_co,
     floattext_simulation_time_mid_co,
     inttext_n_iter_mid_co,
     label_empty_space,
     layout_validate_3_mid_co,
     label_empty_space,
     accordion_4_mid_co])

box_crrid_generic = widgets.VBox(
    [title_analysis,
     description_label_crrid_generic,
     label_empty_space,
     box_input_1_crrid_generic,
     label_empty_space,
     box_input_2_crrid_generic,
     label_empty_space,
     layout_validate_1_crrid_generic,
     label_empty_space,
     accordion_1_crrid_generic,
     separator_label,
     title_time_simulation,
     time_simulation_label,
     label_empty_space,
     stacked_simulation_time_crrid_generic,
     floattext_simulation_time_crrid_generic,
     inttext_n_iter_crrid_generic,
     label_empty_space,
     layout_validate_2_crrid_generic,
     label_empty_space,
     accordion_2_crrid_generic])

box_pdf_report = widgets.VBox(
    [title_pdf_report,
     description_pdf_report_1,
     label_empty_space,
     ck_pdf_report_analysis_mid_generic,
     ck_pdf_report_time_simulation_mid_generic,
     label_empty_space,
     export_pdf_button_mid_generic,
     label_empty_space,
     output_file_link_mid_generic,
     separator_label,
     label_empty_space,
     ck_pdf_report_analysis_mid_co,
     ck_pdf_report_sensitivity_mid_co,
     ck_pdf_report_time_simulation_mid_co,
     label_empty_space,
     export_pdf_button_mid_co,
     label_empty_space,
     output_file_link_mid_co,
     separator_label,
     label_empty_space,
     ck_pdf_report_analysis_crrid_generic,
     ck_pdf_report_time_simulation_crrid_generic,
     label_empty_space,
     export_pdf_button_crrid_generic,
     label_empty_space,
     output_file_link_crrid_generic,])

box_example = widgets.VBox([title_example_1,
                            description_image,
                            separator_label,
                            title_analysis,
                            description_label_2_example_1,
                            label_empty_space,
                            box_input_example_1,
                            label_empty_space,
                            description_label_3_example_1,
                            label_empty_space,
                            vbox_1_limits_example_1,
                            label_empty_space,
                            box_input_2_example_1,
                            label_empty_space,
                            accordion_2_example_1,
                            label_empty_space,
                            description_label_4_example_1,
                            label_empty_space,
                            box_input_3_example_1,
                            label_empty_space,
                            layout_validate_1_example_1,
                            label_empty_space,
                            accordion_1_example_1])

tab = widgets.Tab()
tab.children = [box_home,
                box_mid_generic,
                box_mid_co,
                box_crrid_generic,
                box_pdf_report,
                box_example]

tab.titles = ('Home',
              'Generic MID',
              'Control oriented MID',
              'Generic CRRID',
              'PDF Report',
              'Example')

file = open("images/header_p3d_online.png", "rb")
image = file.read()
header_image = widgets.Image(value=image, format='png', width=975, height=25)




dashboard = widgets.VBox([header_image, tab])
display(dashboard)